In [ ]:
import numpy as np
import pandas as pd
import xarray as xr
import cdo
CDO = cdo.Cdo()
import netCDF4
import os
import datetime
import matplotlib.pyplot as plt 
from matplotlib import colormaps as cm
import cartopy.crs as ccrs
from scipy import stats
from scipy import signal
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly_express as px
import shutil
#from mycolorpy import colorlist as mcp
import xskillscore as xs
import seaborn as sns
from netCDF4 import Dataset,num2date,date2num
import scipy
import iris
#from eofs.iris import Eof
from eofs.xarray import Eof
import xarray_regrid

# Functions

In [ ]:
def detrend_dim(da, dim, deg=1):
    # detrend along a single dimension
    p = da.polyfit(dim=dim, deg=deg)
    fit = xr.polyval(da[dim], p.polyfit_coefficients)
    return da - fit

In [ ]:
def sel_mon(file,mon):
    output=file.where(file.time.dt.month.isin([mon]), drop=True)
    return output

In [ ]:
def create_season_resolution(da,month_or_season):
    # ref_time should have the format ref_time=["1901-01-01","2000-12-31"]
    if isinstance(month_or_season, str):
        print("for seasons to work your data should contain all 12 months in each year")
        ds_seasons=da.resample(time="QS-DEC").mean()
        #print(ds_seasons.shape)
        DJF=ds_seasons.time.dt.month.isin([12])
        DJF=DJF[:-1]
        MAM=ds_seasons.time.dt.month.isin([3])
        JJA=ds_seasons.time.dt.month.isin([6])
        SON=ds_seasons.time.dt.month.isin([9])

        if month_or_season=="MAM":
            ts=ds_seasons.where(MAM,drop=True)
        if month_or_season=="JJA":
            ts=ds_seasons.where(JJA,drop=True)
            #print(ts.shape)
        if month_or_season=="OND":
            ts=ds_seasons.where(OND,drop=True)
        if month_or_season=="DJF":
            ts=ds_seasons.where(DJF,drop=True)
    else:
        ts=sel_mon(da,month_or_season)

    return ts


In [ ]:
def create_anomalies(da,month_or_season,ref_time):
    # ref_time should have the format ref_time=["1901-01-01","2000-12-31"]
    if isinstance(month_or_season, str):
        print("for seasons to work your data should contain all 12 months in each year")
        ds_seasons=da.resample(time="QS-DEC").mean()
        #print(ds_seasons.shape)
        DJF=ds_seasons.time.dt.month.isin([12])
        DJF=DJF[:-1]
        MAM=ds_seasons.time.dt.month.isin([3])
        JJA=ds_seasons.time.dt.month.isin([6])
        SON=ds_seasons.time.dt.month.isin([9])

    
        if month_or_season=="MAM":
            ts=ds_seasons.where(MAM,drop=True)
        if month_or_season=="JJA":
            ts=ds_seasons.where(JJA,drop=True)
            #print(ts.shape)
        if month_or_season=="OND":
            ts=ds_seasons.where(OND,drop=True)
        if month_or_season=="DJF":
            ts=ds_seasons.where(DJF,drop=True)

        climate=da.sel(time=slice(ref_time[0],ref_time[1]))
        seasonmean_climate=season_mean(climate)
        seasonmean_climate=seasonmean_climate.sel(season=month_or_season)
        anom=ts-seasonmean_climate.squeeze()
    else:
        climate=da.sel(time=slice(ref_time[0],ref_time[1]))
        mon=sel_mon(da,month_or_season)
        mon_climate=sel_mon(climate,month_or_season)
        mon_climate=mon_climate.mean(dim="time")
        anom=mon-mon_climate
    return anom

In [ ]:
def create_anomalies_alt(da,month_or_season,ref_time):
    # ref_time should have the format ref_time=["1901-01-01","2000-12-31"]
    if isinstance(month_or_season, str):
        print("for seasons to work your data should contain all 12 months in each year")
        ds_seasons=da.resample(time="QS-DEC").mean()
        #print(ds_seasons.shape)
        DJF=ds_seasons.time.dt.month.isin([12])
        DJF=DJF[:-1]
        MAM=ds_seasons.time.dt.month.isin([3])
        JJA=ds_seasons.time.dt.month.isin([6])
        SON=ds_seasons.time.dt.month.isin([9])

    
        if month_or_season=="MAM":
            ts=ds_seasons.where(MAM,drop=True)
        if month_or_season=="JJA":
            ts=ds_seasons.where(JJA,drop=True)
            #print(ts.shape)
        if month_or_season=="OND":
            ts=ds_seasons.where(OND,drop=True)
        if month_or_season=="DJF":
            ts=ds_seasons.where(DJF,drop=True)

        climate=ts.sel(time=slice(ref_time[0],ref_time[1]))
        seasonmean_climate=climate.mean(dim="time")
        anom=ts-seasonmean_climate.squeeze()
    else:
        climate=da.sel(time=slice(ref_time[0],ref_time[1]))
        mon=sel_mon(da,month_or_season)
        mon_climate=sel_mon(climate,month_or_season)
        mon_climate=mon_climate.mean(dim="time")
        anom=mon-mon_climate
    return anom

In [ ]:
def corr_with_sig(da_A,da_B,sig_level,lon_name="longitude",lat_name="latitude"):
    timesteps=da_A.shape[0]
    Nlatitudes=da_A.shape[1]
    Nlongitudes=da_A.shape[2]
    
    if da_A.shape==da_B.shape:
        print("dimensions align")
    else:
        print("your data sets need to have the same dimensions")
    
    da_A_4_corr=da_A.values.reshape(timesteps,Nlatitudes*Nlongitudes)
    da_B_4_corr=da_B.values.reshape(timesteps,Nlatitudes*Nlongitudes)
    
    structure_R= np.arange(Nlatitudes*Nlongitudes, dtype=float)
    structure_R_masked= np.arange(Nlatitudes*Nlongitudes, dtype=float)
    structure_p=np.arange(Nlatitudes*Nlongitudes, dtype=float)
    
    for a in range(Nlatitudes*Nlongitudes):
        one_R=stats.pearsonr(da_A_4_corr[:,a], da_B_4_corr[:,a])
        structure_R[a]=one_R[0]
        if one_R[1] <= sig_level:
            structure_R_masked[a]=one_R[0]
        else:
            structure_R_masked[a] = np.ma.masked_greater(2, sig_level)
        structure_p[a]=one_R[1]
    R_grid=structure_R.reshape(Nlatitudes,Nlongitudes)
    R_grid_masked=structure_R_masked.reshape(Nlatitudes,Nlongitudes)
    p_grid=structure_p.reshape(Nlatitudes,Nlongitudes)
    
    xr_R = xr.DataArray(
        data=R_grid,
        dims=[lat_name,lon_name],
        coords=dict(
            longitude=da_A[lon_name],
            latitude=da_A[lat_name],),
        attrs=dict(
            description="Corr Coef",),)
    
    xr_R_masked = xr.DataArray(
        data=R_grid_masked,
        dims=[lat_name,lon_name],
        coords=dict(
            longitude=da_A[lon_name],
            latitude=da_A[lat_name],),
        attrs=dict(
            description="Corr Coef masked "+str(sig_level),),)
    
    xr_p = xr.DataArray(
        data=p_grid,
        dims=[lat_name,lon_name],
        coords=dict(
            longitude=da_A[lon_name],
            latitude=da_A[lat_name],),
        attrs=dict(
            description="P-Value",),)
    return xr_R,xr_R_masked,xr_p 


In [ ]:
# Wrap it into a simple function
def season_mean(ds, calendar="standard"):
    # Make a DataArray with the number of days in each month, size = len(time)
    month_length = ds.time.dt.days_in_month

    # Calculate the weights by grouping by 'time.season'
    weights = (
        month_length.groupby("time.season") / month_length.groupby("time.season").sum()
    )

    # Test that the sum of the weights for each season is 1.0
    np.testing.assert_allclose(weights.groupby("time.season").sum().values, np.ones(4))

    # Calculate the weighted average
    return (ds * weights).groupby("time.season").sum(dim="time")

In [ ]:
def enlarge_grid(array,da,description,lon_name="longitude",lat_name="latitude"):
    da_shape=da.values.shape
    array=array.squeeze()
    empty_grid=np.zeros_like(da.values)
    for i in range(empty_grid.shape[0]):
        empty_grid[i,:,:]=np.repeat(array[i],empty_grid.shape[1]*empty_grid.shape[2]).reshape(empty_grid.shape[1],empty_grid.shape[2])
    new_da = xr.DataArray(
    data=empty_grid,
    dims=["time", lat_name,lon_name],
    coords=da.coords,
    attrs=dict(
        description=description,),)
    return new_da

In [ ]:
current_cmap_rain = cm.get_cmap("BrBG")
current_cmap_rain.set_bad(color='white')

In [ ]:
current_cmap_v = cm.get_cmap("PuOr")
current_cmap_v.set_bad(color='white')

In [ ]:
current_cmap_t = cm.get_cmap("bwr")
current_cmap_t.set_bad(color='white')

In [ ]:
current_cmap_olr = cm.get_cmap("RdGy_r")
current_cmap_olr.set_bad(color='white')

In [ ]:
current_cmap_z = cm.get_cmap("PiYG")
current_cmap_z.set_bad(color='white')

In [ ]:
current_cmap = cm.get_cmap("seismic")
current_cmap.set_bad(color='white')

# Author

Name: Martin Wegmann

Date: November 2025

Email: martin.wegmann@unibe.ch

# Data sources

see data availability statement of corresponding paper

# Folders

In [ ]:
#
plot_folder="/Volumes/SPARK/o18/2024/plots/"
input_folder="/Volumes/SPARK/o18/"
input_folder_2024="/Volumes/SPARK/o18/2024/"

tcr_folder="/Volumes/SPARK/20crv3/"
era20c_folder="/Volumes/SPARK/era20c/"
cera20c_folder="/Volumes/SPARK/cera20c/"
asf20c_folder="/Volumes/SPARK/afs20c/"
csf20c_folder="/Volumes/SPARK/cfs20c/"
ekf_folder="/Volumes/SPARK/ekf400v2/ensmean/"
paleora_folder="/Volumes/SPARK3/ModE-RA/"
do18_folder="/Volumes/SPARK/o18/"
era5_folder="/Volumes/SPARK3/era5/o18/"

sst_folder="/Volumes/SPARK/paleora_sst/"
hadisst_folder="/Volumes/SPARK/hadisst2/"

# Grid Preprocessing

First thing we have to do is to unify the coordinates of all our gridded products

We use mostly CDO in the shell for this, even though the mix between python and shell is messy and sometimes you have to correct manually some folder locations if you want to reproduce this

In [ ]:
os.chdir(ekf_folder)
!cdo sellonlatbox,-180,180,-90,90 EKF400_ensmean_v2.0.nc  EKF400_ensmean_v2.0_remapped.nc 

In [ ]:
os.chdir(era5_folder)
!cdo -b 32 sellonlatbox,-180,180,-90,90 era5_v200.nc era5_v200_remapped.nc 
!cdo -b 32 sellonlatbox,-180,180,-90,90 era5_slp.nc era5_slp_remapped.nc 
!cdo -b 32 sellonlatbox,-180,180,-90,90 era5_summer_monthly.nc era5_summer_monthly_remapped.nc 


In [ ]:
os.chdir(tcr_folder)
!cdo -L -b 32 sellonlatbox,-180,180,-90,90 -selname,air air.2m.mon.mean.nc  air.2m.mon.mean_remapped.nc 
!cdo -L -b 32  sellonlatbox,-180,180,-90,90 -selname,prmsl prmsl.mon.mean.nc  prmsl.mon.mean_remapped.nc 
!cdo -L -b 32  sellonlatbox,-180,180,-90,90 -selname,prate prate.mon.mean.nc  prate.mon.mean_remapped.nc 
!cdo -L -b 32  sellonlatbox,-180,180,-90,90 -selname,uwnd uwnd.mon.mean.nc  uwnd.mon.mean_remapped.nc
!cdo -L -b 32  sellonlatbox,-180,180,-90,90 -selname,vwnd vwnd.mon.mean.nc vwnd.mon.mean_remapped.nc

create modera ensmean

In [ ]:
os.chdir(paleora_folder)
!cdo -L -b 32 ensmean ModE-RA_m0*_slp_anom_wrt_1901-2000_1421-2008_mon.nc ModE-RA_ensmean_slp_anom_wrt_1901-2000_1421-2008_mon.nc
!cdo -L -b 32 ensmean ModE-RA_m0*_temp2_anom_wrt_1901-2000_1421-2008_mon.nc ModE-RA_ensmean_temp2_anom_wrt_1901-2000_1421-2008_mon.nc
!cdo -L -b 32 ensmean ModE-RA_m0*_totprec_anom_wrt_1901-2000_1421-2008_mon.nc ModE-RA_ensmean_totprec_anom_wrt_1901-2000_1421-2008_mon.nc
!cdo -L -b 32 ensmean ModE-RA_m0*_geopoth_50000_anom_wrt_1901-2000_1421-2008_mon.nc ModE-RA_ensmean_geopoth_50000_anom_wrt_1901-2000_1421-2008_mon.nc


In [ ]:
os.chdir(paleora_folder)
!cdo -L -b 32 sellonlatbox,-180,180,-90,90 ModE-RA_ensmean_slp_anom_wrt_1901-2000_1421-2008_mon.nc ModE-RA_ensmean_slp_anom_wrt_1901-2000_1421-2008_mon_remapped.nc
!cdo -L -b 32 sellonlatbox,-180,180,-90,90 ModE-RA_ensmean_temp2_anom_wrt_1901-2000_1421-2008_mon.nc ModE-RA_ensmean_temp2_anom_wrt_1901-2000_1421-2008_mon_remapped.nc
!cdo -L -b 32 sellonlatbox,-180,180,-90,90 ModE-RA_ensmean_totprec_anom_wrt_1901-2000_1421-2008_mon.nc ModE-RA_ensmean_totprec_anom_wrt_1901-2000_1421-2008_mon_remapped.nc
!cdo -L -b 32 sellonlatbox,-180,180,-90,90 ModE-RA_ensmean_geopoth_50000_anom_wrt_1901-2000_1421-2008_mon.nc ModE-RA_ensmean_geopoth50ß_anom_wrt_1901-2000_1421-2008_mon_remapped.nc


CERA20C and ERA20C are already on the -180,180 coordinate system

SSTs

In [ ]:
os.chdir(sst_folder)

In [ ]:
#postprocessing ERICS SSTs
!for S in $(seq -w 001 050);do echo $S; cdo -L -b 32 selyear,1600/1849 -selname,sst PaleoSST_SIC_1000-1849_R0${S}.nc PaleoSST_1600-1849_R0${S}.nc; cdo yearmean -selmon,4,5 PaleoSST_1600-1849_R0${S}.nc PaleoSST_1600-1849_R0${S}_AM.nc;  done

In [ ]:
!cdo ensmean PaleoSST_1600-1849_R0*_AM.nc PaleoSST_1600-1849_ensmean_AM.nc

In [ ]:
!cdo remapbil,/Volumes/SPARK/ekf400v2/ensmean/EKF400_ensmean_v2.0_remapped.nc PaleoSST_1600-1849_ensmean_AM.nc PaleoSST_1600-1849_ensmean_AM_remap.nc

In [ ]:
!for S in $(seq -w 001 050);do echo $S; cdo remapbil,/Volumes/SPARK/ekf400v2/ensmean/EKF400_ensmean_v2.0_remapped.nc PaleoSST_1600-1849_R0${S}_AM.nc PaleoSST_1600-1849_R0${S}_AM_remap.nc;  done

In [ ]:

os.chdir(hadisst_folder)

In [ ]:
!cdo ensmean HadISST.2.1.0.0_realisation* HadISST.2.1.0.0_ensmean.nc 

In [ ]:
!cdo remapbil,/Volumes/SPARK/ekf400v2/ensmean/EKF400_ensmean_v2.0_remapped.nc -yearmean -selmon,4,5 -selname,sst HadISST.2.1.0.0_ensmean.nc HadISST.2.1.0.0_ensmean_AM.nc

In [ ]:
os.chdir(sst_folder)

In [ ]:
!cdo -O mergetime /Volumes/SPARK/hadisst2/HadISST.2.1.0.0_ensmean_AM.nc PaleoSST_1600-1849_ensmean_AM_remap.nc PaleoSST_1600-2010_mergensmean_AM_remap.nc

# read in do18 network information

This data is re-used from Treydte et al 2024

first we get the coordinates of the trees

In [ ]:
new_coords=pd.read_csv(input_folder_2024+"coords.txt",sep=";")

In [ ]:
new_coords

In [ ]:
lats_new=pd.to_numeric(new_coords['latitude'], errors='coerce').values
lons_new=pd.to_numeric(new_coords['longitude'], errors='coerce').values

getting the eof information

for computation of EOF information see script "EOF_computation.R"

This data is provided by Daniel Balting

In [ ]:
pcs=pd.read_csv(input_folder_2024+"PC_ts.csv",sep=";")

In [ ]:
pcs

In [ ]:
pc1df=pd.DataFrame({"PC1_o18_new":pcs.PC1.values}, index=pd.period_range('1600-7-16', '2013-7-16',freq="y"))


In [ ]:
pc2df=pd.DataFrame({"PC2_o18_new":pcs.PC2.values}, index=pd.period_range('1600-7-16', '2013-7-16',freq="y"))


In [ ]:
pc1df_short=pc1df["16000101":"20050101"]
pc2df_short=pc2df["16000101":"20050101"]

In [ ]:
pc1df_ekf=pc1df["16020101":"20031231"]
pc1df_tcr=pc1df["18360101":"20131231"]
pc1df_cera20c=pc1df["19010101":"20101231"]
pc1df_modera=pc1df["16000101":"20081231"]
pc1df_ssts=pc1df["16000101":"20101231"]


comparison with older PC time series based on less locations

In [ ]:
pc1=pd.read_csv(input_folder+"PC_O18_1600_2005_PC1.txt",header=None)
pc1_df=pd.DataFrame({"PC1_o18":pc1.values[:,0]}, index=pd.period_range('1600-7-16', '2005-7-16',freq="y"))



In [ ]:
plt.figure(figsize=(10, 5), dpi= 300)
pc1_df.plot(xlabel='Time', ylabel='Eigenvalues', title='o18 PC1',grid=1,style='r-',colormap="viridis", ax = plt.gca(), alpha=0.8,label="old",legend=True)
pc1df.plot(xlabel='Time', ylabel='Eigenvalues', title='o18 PC1 ',grid=1,style='b-',colormap="viridis", ax = plt.gca(), alpha=0.8,label="new",legend=True)
plt.title("Correlation: "+str(pc1_df['PC1_o18'].corr(pc1df['PC1_o18_new'])))
plt.savefig(plot_folder+"PC1_correlation_newvsold.pdf")

In [ ]:
plt.figure(figsize=(10, 5), dpi= 300)
#pc1_df.plot(xlabel='Time', ylabel='Eigenvalues', title='o18 PC1',grid=1,style='r-',colormap="viridis", ax = plt.gca(), alpha=0.8,label="old",legend=True)
pc1df.plot(xlabel='Time', ylabel='Eigenvalues', title='o18 PC1 ',grid=1,style='b-',colormap="viridis", ax = plt.gca(), alpha=0.8,label="new",legend=True)
#plt.title("Correlation: "+str(pc1_df['PC1_o18'].corr(pc1df['PC1_o18_new'])))
plt.savefig(plot_folder+"PC1_correlation_new.pdf")

In [ ]:
pc1df.to_csv(input_folder_2024+"PC_O18_1600_2005_PC1_new.txt",header=False, index=False)

In [ ]:
pc1_df.to_csv(input_folder_2024+"PC_O18_1600_2005_PC1_old.txt",header=False, index=False)

# creation of PC1 tree ring NC files
Since xarray is a bit sluggish and annoying with the time axis, we use CDO in shell. The goal here is to go from .csv to .txt to .nc without xarray. 

In [ ]:
os.chdir(input_folder_2024)

In [ ]:
# create netcdf files from text
!cdo -f nc input,r1x1 o18_PC1_python_old.nc < PC_O18_1600_2005_PC1_old.txt
!cdo -f nc input,r1x1 o18_PC1_python_new.nc < PC_O18_1600_2005_PC1_new.txt

In [ ]:
# set time axis for our new netcdf files
!cdo -r -chname,var1,o18 -settaxis,1600-01-01,00:00:00,1year o18_PC1_python_old.nc o18_PC1_old.nc
!cdo -r -chname,var1,o18 -settaxis,1600-01-01,00:00:00,1year o18_PC1_python_new.nc o18_PC1_new.nc

In [ ]:
# in case we need to detrend it
!cdo detrend o18_PC1_old.nc o18_PC1_detrend_old.nc
!cdo detrend o18_PC1_new.nc o18_PC1_detrend_new.nc

lets check out our new data

In [ ]:
os.chdir(input_folder_2024)

In [ ]:
o18_PC1_old_detrend=xr.open_dataset("o18_PC1_detrend_old.nc",decode_times=False)

o18_PC1_old_detrend=o18_PC1_old_detrend.o18[:,0,0]

In [ ]:
o18_PC1_new_detrend=xr.open_dataset("o18_PC1_detrend_new.nc",decode_times=False)

o18_PC1_new_detrend=o18_PC1_new_detrend.o18[:,0,0]

In [ ]:
o18_PC1_new_non_detrend=xr.open_dataset("o18_PC1_new.nc",decode_times=False)

o18_PC1_new_non_detrend=o18_PC1_new_non_detrend.o18[:,0,0]

In [ ]:
o18_PC1_old_detrend=pd.DataFrame({"PC1_o18_detrend":o18_PC1_old_detrend.values.reshape(406)}, index=pd.period_range('1600-7-16', '2005-7-16',freq="y"))
o18_PC1_new_detrend=pd.DataFrame({"PC1_o18_detrend_new":o18_PC1_new_detrend.values}, index=pd.period_range('1600-7-16', '2013-7-16',freq="y"))
o18_PC1_new_non_detrend=pd.DataFrame({"PC1_o18_non_detrend_new":o18_PC1_new_non_detrend.values}, index=pd.period_range('1600-7-16', '2013-7-16',freq="y"))



In [ ]:
plt.figure(figsize=(10, 5), dpi= 300)
o18_PC1_old_detrend.plot(xlabel='Time', ylabel='Eigenvalues', title='o18 PC1 detrend',grid=1,style='r-',colormap="viridis", ax = plt.gca(), alpha=1,legend=True)
o18_PC1_new_detrend.plot(xlabel='Time', ylabel='Eigenvalues', title='o18 PC1 detrend',grid=1,style='b-',colormap="viridis", ax = plt.gca(), alpha=1,legend=True)
plt.title("Correlation: "+str(o18_PC1_old_detrend['PC1_o18_detrend'].corr(o18_PC1_new_detrend['PC1_o18_detrend_new'])))


plt.savefig(plot_folder+"PC1_correlation_newvsold_detrend.pdf")

In [ ]:
plt.figure(figsize=(10, 5), dpi= 300)
#o18_PC1_old_detrend.plot(xlabel='Time', ylabel='Eigenvalues', title='o18 PC1 detrend',grid=1,style='r-',colormap="viridis", ax = plt.gca(), alpha=1,legend=True)
o18_PC1_new_detrend.plot(xlabel='Time', ylabel='Eigenvalues', title='o18 PC1 detrend',grid=1,style='b-',colormap="viridis", ax = plt.gca(), alpha=1,legend=True)
plt.title("Correlation: "+str(o18_PC1_old_detrend['PC1_o18_detrend'].corr(o18_PC1_new_detrend['PC1_o18_detrend_new'])))


plt.savefig(plot_folder+"PC1_correlation_new_detrend.pdf")

In [ ]:
plt.figure(figsize=(10, 5), dpi= 300)
#o18_PC1_old_detrend.plot(xlabel='Time', ylabel='Eigenvalues', title='o18 PC1 detrend',grid=1,style='r-',colormap="viridis", ax = plt.gca(), alpha=1,legend=True)
o18_PC1_new_non_detrend.plot(xlabel='Time', ylabel='Eigenvalues', title='o18 PC1 non_detrend',grid=1,style='r-',ylim=(-11,11),colormap="viridis", ax = plt.gca(), alpha=1,legend=True)
o18_PC1_new_detrend.plot(xlabel='Time', ylabel='Eigenvalues', title='o18 PC1 detrend',grid=1,style='b-',ylim=(-11,11),colormap="viridis", ax = plt.gca(), alpha=1,legend=True)
o18_PC1_new_detrend[o18_PC1_new_detrend.index=="2003"].plot(xlabel='Time', ylabel='Eigenvalues', title='o18 PC1 detrend',grid=1,style='b.',ylim=(-11,11),markersize=20,colormap="viridis", ax = plt.gca(), alpha=0.7,legend=False)
o18_PC1_new_detrend[o18_PC1_new_detrend.index=="1816"].plot(xlabel='Time', ylabel='Eigenvalues', title='o18 PC1 detrend',grid=1,style='b.',ylim=(-11,11),markersize=20,colormap="viridis", ax = plt.gca(), alpha=0.7,legend=False)

plt.title("Correlation: "+str(o18_PC1_new_non_detrend['PC1_o18_non_detrend_new'].corr(o18_PC1_new_detrend['PC1_o18_detrend_new'])))
plt.text("1994", -7.7, '2003', fontsize = 10,color="blue")
plt.text("1807", 9.5, '1816', fontsize = 10,color="blue")

#plt.plot("1620", 7, 'ro')


plt.savefig(plot_folder+"PC1_correlation_new_non_events_detrend.pdf")

In [ ]:
plt.figure(figsize=(10, 5), dpi= 300)
#o18_PC1_old_detrend.plot(xlabel='Time', ylabel='Eigenvalues', title='o18 PC1 detrend',grid=1,style='r-',colormap="viridis", ax = plt.gca(), alpha=1,legend=True)
o18_PC1_new_non_detrend.plot(xlabel='Time', ylabel='Eigenvalues', title='o18 PC1 non_detrend',grid=1,style='r-',colormap="viridis", ax = plt.gca(), alpha=1,legend=False)
o18_PC1_new_detrend.plot(xlabel='Time', ylabel='Eigenvalues', title='o18 PC1 detrend',grid=1,style='b-',colormap="viridis", ax = plt.gca(), alpha=1,legend=False)
o18_PC1_new_detrend[o18_PC1_new_detrend.index=="2003"].plot(xlabel='Time', ylabel='Eigenvalues', title='o18 PC1 detrend',grid=1,style='b.',markersize=20,colormap="viridis", ax = plt.gca(), alpha=1,legend=False)
o18_PC1_new_detrend[o18_PC1_new_detrend.index=="1816"].plot(xlabel='Time', ylabel='Eigenvalues', title='o18 PC1 detrend',grid=1,style='b.',markersize=20,colormap="viridis", ax = plt.gca(), alpha=1,legend=False)

plt.title("Correlation: "+str(o18_PC1_new_non_detrend['PC1_o18_non_detrend_new'].corr(o18_PC1_new_detrend['PC1_o18_detrend_new'])))
plt.text("1994", -7.7, '2003', fontsize = 10,color="blue")
plt.text("1807", 9, '1816', fontsize = 10,color="blue")

#plt.plot("1620", 7, 'ro')


plt.savefig(plot_folder+"PC1_correlation_new_non_events_detrend_noleg.pdf")

In [ ]:
extreme_ears_marks=o18_PC1_new_detrend.values<-6.4

In [ ]:
wet_ears_marks=o18_PC1_new_detrend.values>5.5

In [ ]:
o18_PC1_new_detrend[o18_PC1_new_detrend.index=="2003"]

In [ ]:
o18_PC1_new_detrend[o18_PC1_new_detrend.index=="1816"]

In [ ]:
o18_PC1_new_detrend.index.values[wet_ears_marks.squeeze()]

In [ ]:
o18_PC1_new_detrend.index.values[extreme_ears_marks.squeeze()]

In [ ]:
o18_PC1_new_detrend.loc["2003"]

In [ ]:
o18_PC1_new_detrend.loc["2013"]

In [ ]:
# enlarge it to EKF400 grid
!cdo enlarge,/Volumes/SPARK/ekf400v2/ensmean/EKF400_ensmean_v2.0_remapped.nc o18_PC1_detrend_old.nc o18_PC1_detrend_enlarge_old.nc
!cdo enlarge,/Volumes/SPARK/ekf400v2/ensmean/EKF400_ensmean_v2.0_remapped.nc o18_PC1_detrend_new.nc o18_PC1_detrend_enlarge_new.nc
!cdo enlarge,/Volumes/SPARK/ekf400v2/ensmean/EKF400_ensmean_v2.0_remapped.nc -selyear,1600/2008 o18_PC1_detrend_new.nc o18_PC1_detrend_enlarge_new_short.nc

# EAPattern EOF

## modera

In [ ]:
lon_list_EAPdomain_modera= np.arange(-70, 40+1.875, 1.875)
lat_list_EAPdomain_modera = np.arange(25, 80+1.875, 1.875)

In [ ]:
moderea_slp=xr.open_dataset(paleora_folder+"ModE-RA_ensmean_slp_anom_wrt_1901-2000_1421-2008_mon_remapped.nc")

In [ ]:
moderea_ensmean_JJA_slp_anoms1901=create_season_resolution(da=moderea_slp.slp,month_or_season="JJA")

In [ ]:
moderea_ensmean_JJA_slp_anoms1901_EAP=moderea_ensmean_JJA_slp_anoms1901.sel(longitude=lon_list_EAPdomain_modera,latitude=lat_list_EAPdomain_modera,method="nearest")

In [ ]:
moderea_ensmean_JJA_slp_anoms1901_EAP=moderea_ensmean_JJA_slp_anoms1901_EAP.sel(time=slice("1941","2003"))

In [ ]:
coslat = np.cos(np.deg2rad(moderea_ensmean_JJA_slp_anoms1901_EAP.coords['latitude'].values))
wgts = np.sqrt(coslat)[..., np.newaxis]
# read a spatial-temporal field, time must be the first dimension
#eofs_ekf400_monthly = iris.load_cube("/Volumes/SPARK/ekf400v2/ensmean/EKF400_ensmean_v2.0_JJA_v200_anoms1951_CGTdomain.nc")
# create a solver class, taking advantage of built-in weighting
eofs_monthly  = Eof(moderea_ensmean_JJA_slp_anoms1901_EAP , weights=wgts)
variance_fractions_monthly = eofs_monthly.varianceFraction(neigs=3)
pcs_EAP_allyears = eofs_monthly.pcs(npcs=3, pcscaling=1)

# retrieve the first two EOFs from the solver class
eofs_monthly  = eofs_monthly.eofs(neofs=3)
#xr_eofs_ekf400_monthly =xr.DataArray.from_iris(eofs_ekf400_monthly)


In [ ]:
fig = plt.figure(figsize=(10, 5), dpi= 300)
ax = plt.axes(projection=ccrs.EqualEarth(central_longitude=0, globe=None)) 
tplot=eofs_monthly[0,:,:].plot.pcolormesh(ax=ax,
levels = 17, transform=ccrs.PlateCarree(), cmap=current_cmap, cbar_kwargs={'orientation':'horizontal',
'fraction':0.042, 'pad':0.015, 'aspect':35})

tplot.colorbar.set_label('eigenvalues', size=10) 
tplot.ylabel_style = {'size':16}

ax.set_global()
ax.coastlines()
ax.set_title('JJA anoms SLP First EOF ModE-RA ensmean wrt 1901-2000')
ax.set_extent([-95, 95, 20, 80])
;
fig.savefig(plot_folder+"JJAslp_1stEOFmodera19012000_19412003.pdf")

In [ ]:
fig = plt.figure(figsize=(10, 5), dpi= 300)
ax = plt.axes(projection=ccrs.EqualEarth(central_longitude=0, globe=None)) 
tplot=eofs_monthly[1,:,:].plot.pcolormesh(ax=ax,
levels = 17, transform=ccrs.PlateCarree(), cmap=current_cmap, cbar_kwargs={'orientation':'horizontal',
'fraction':0.042, 'pad':0.015, 'aspect':35})

tplot.colorbar.set_label('eigenvalues', size=10) 
tplot.ylabel_style = {'size':16}

ax.set_global()
ax.coastlines()
ax.set_title('JJA anoms SLP Second EOF ModE-RA ensmean wrt 1901-2000')
ax.set_extent([-95, 95, 20, 80])
;
fig.savefig(plot_folder+"JJAslp_2ndtEOFmodera19012000_19412003.pdf")

## EKF400v2

In [ ]:

lon_list_EAPdomain_ekf= np.arange(-70, 40+1.875, 1.875)
lat_list_EAPdomain_ekf = np.arange(25, 80+1.875, 1.875)

In [ ]:
ekf=xr.open_dataset(ekf_folder+"EKF400_ensmean_v2.0_remapped.nc")

ekf["time"]=xr.cftime_range(start = '1602-01-01', end = '2003-12-31',freq="M",calendar = 'standard')


In [ ]:
EKF400_ensmean_JJA_slp_anoms1951=create_anomalies(da=ekf.air_pressure_at_sea_level,month_or_season="JJA",ref_time=["1951-01-01","1980-12-31"])


In [ ]:
EKF400_ensmean_JJA_slp_anoms1951_EAP=EKF400_ensmean_JJA_slp_anoms1951.sel(longitude=lon_list_EAPdomain_ekf,latitude=lat_list_EAPdomain_ekf,method="nearest")

In [ ]:
EKF400_ensmean_JJA_slp_anoms1951_EAP=EKF400_ensmean_JJA_slp_anoms1951_EAP.sel(time=slice("1941","2003"))

In [ ]:
coslat = np.cos(np.deg2rad(EKF400_ensmean_JJA_slp_anoms1951_EAP.coords['latitude'].values))
wgts = np.sqrt(coslat)[..., np.newaxis]
# read a spatial-temporal field, time must be the first dimension
#eofs_ekf400_monthly = iris.load_cube("/Volumes/SPARK/ekf400v2/ensmean/EKF400_ensmean_v2.0_JJA_v200_anoms1951_CGTdomain.nc")
# create a solver class, taking advantage of built-in weighting
eofs_monthly  = Eof(EKF400_ensmean_JJA_slp_anoms1951_EAP , weights=wgts)
variance_fractions_monthly = eofs_monthly.varianceFraction(neigs=3)
pcs_EAP_allyears = eofs_monthly.pcs(npcs=3, pcscaling=1)

# retrieve the first two EOFs from the solver class
eofs_monthly  = eofs_monthly.eofs(neofs=3)
#xr_eofs_ekf400_monthly =xr.DataArray.from_iris(eofs_ekf400_monthly)


In [ ]:
fig = plt.figure(figsize=(10, 5), dpi= 300)
ax = plt.axes(projection=ccrs.EqualEarth(central_longitude=0, globe=None)) 
tplot=eofs_monthly[0,:,:].plot.pcolormesh(ax=ax,
levels = 17, transform=ccrs.PlateCarree(), cmap=current_cmap, cbar_kwargs={'orientation':'horizontal',
'fraction':0.042, 'pad':0.015, 'aspect':35})

tplot.colorbar.set_label('eigenvalues', size=10) 
tplot.ylabel_style = {'size':16}

ax.set_global()
ax.coastlines()
ax.set_title('JJA anoms SLP First EOF EKF400v2 ensmean wrt 1951-1980')
ax.set_extent([-95, 95, 20, 80])
;
fig.savefig(plot_folder+"JJAslp_1stEOFekf19511980_19412003.pdf")

In [ ]:
fig = plt.figure(figsize=(10, 5), dpi= 300)
ax = plt.axes(projection=ccrs.EqualEarth(central_longitude=0, globe=None)) 
tplot=eofs_monthly[1,:,:].plot.pcolormesh(ax=ax,
levels = 17, transform=ccrs.PlateCarree(), cmap=current_cmap, cbar_kwargs={'orientation':'horizontal',
'fraction':0.042, 'pad':0.015, 'aspect':35})

tplot.colorbar.set_label('eigenvalues', size=10) 
tplot.ylabel_style = {'size':16}

ax.set_global()
ax.coastlines()
ax.set_title('JJA anoms SLP Second EOF EKF400v2 ensmean wrt 1951-1980')
ax.set_extent([-95, 95, 20, 80])
;
fig.savefig(plot_folder+"JJAslp_2ndtEOFekf19511980_19412003.pdf")

## 20CRv3

In [ ]:

lon_list_EAPdomain_tcr= np.arange(-70, 40+1, 1)
lat_list_EAPdomain_tcr = np.arange(25, 80+1, 1)

In [ ]:
tcrv3_slp=xr.open_dataset(tcr_folder+"prmsl.mon.mean_remapped.nc").prmsl


In [ ]:
tcrv3_ensmean_JJA_slp_anoms1951=create_anomalies(da=tcrv3_slp,month_or_season="JJA",ref_time=["1951-01-01","1980-12-31"])


In [ ]:
tcrv3_ensmean_JJA_slp_anoms1951_EAP=tcrv3_ensmean_JJA_slp_anoms1951.sel(lon=lon_list_EAPdomain_tcr,lat=lat_list_EAPdomain_tcr,method="nearest")

In [ ]:
tcrv3_ensmean_JJA_slp_anoms1951_EAP=tcrv3_ensmean_JJA_slp_anoms1951_EAP.sel(time=slice("1941","2003"))

In [ ]:
coslat = np.cos(np.deg2rad(tcrv3_ensmean_JJA_slp_anoms1951_EAP.coords['lat'].values))
wgts = np.sqrt(coslat)[..., np.newaxis]
# read a spatial-temporal field, time must be the first dimension
#eofs_ekf400_monthly = iris.load_cube("/Volumes/SPARK/ekf400v2/ensmean/EKF400_ensmean_v2.0_JJA_v200_anoms1951_CGTdomain.nc")
# create a solver class, taking advantage of built-in weighting
eofs_monthly  = Eof(tcrv3_ensmean_JJA_slp_anoms1951_EAP , weights=wgts)
variance_fractions_monthly = eofs_monthly.varianceFraction(neigs=3)
pcs = eofs_monthly.pcs(npcs=3, pcscaling=1)

# retrieve the first two EOFs from the solver class
eofs_monthly  = eofs_monthly.eofs(neofs=3)
#xr_eofs_ekf400_monthly =xr.DataArray.from_iris(eofs_ekf400_monthly)


In [ ]:
fig = plt.figure(figsize=(10, 5), dpi= 300)
ax = plt.axes(projection=ccrs.EqualEarth(central_longitude=0, globe=None)) 
tplot=eofs_monthly[0,:,:].plot.pcolormesh(ax=ax,
levels = 17, transform=ccrs.PlateCarree(), cmap=current_cmap, cbar_kwargs={'orientation':'horizontal',
'fraction':0.042, 'pad':0.015, 'aspect':35})

tplot.colorbar.set_label('eigenvalues', size=10) 
tplot.ylabel_style = {'size':16}

ax.set_global()
ax.coastlines()
ax.set_title('JJA anoms SLP First EOF 20CRv3 ensmean wrt 1951-1980')
ax.set_extent([-95, 95, 20, 80])
;
fig.savefig(plot_folder+"JJAslp_1stEOFtcr19511980_19412003.pdf")

In [ ]:
fig = plt.figure(figsize=(10, 5), dpi= 300)
ax = plt.axes(projection=ccrs.EqualEarth(central_longitude=0, globe=None)) 
tplot=eofs_monthly[1,:,:].plot.pcolormesh(ax=ax,
levels = 17, transform=ccrs.PlateCarree(), cmap=current_cmap, cbar_kwargs={'orientation':'horizontal',
'fraction':0.042, 'pad':0.015, 'aspect':35})

tplot.colorbar.set_label('eigenvalues', size=10) 
tplot.ylabel_style = {'size':16}

ax.set_global()
ax.coastlines()
ax.set_title('JJA anoms SLP Second EOF 20CRv3 ensmean wrt 1951-1980')
ax.set_extent([-95, 95, 20, 80])
;
fig.savefig(plot_folder+"JJAslp_2ndtEOFtcr19511980_19412003.pdf")

## ERA5

In [ ]:
lon_list_EAPdomain_era5= np.arange(-70, 40+0.25, 0.25)
lat_list_EAPdomain_era5 = np.arange(25, 80+0.25, 0.25)

In [ ]:
era5_slp=xr.open_dataset(era5_folder+"era5_slp_remapped.nc")

In [ ]:
era5_slp_anoms1951=create_anomalies(da=era5_slp.msl,month_or_season="JJA",ref_time=["1951-01-01","1980-12-31"])


In [ ]:
era5_slp_anoms1951_eap=era5_slp_anoms1951.sel(longitude=lon_list_EAPdomain_era5,latitude=lat_list_EAPdomain_era5 ,method="nearest")

In [ ]:
era5_slp_anoms1951_eap=era5_slp_anoms1951_eap.sel(time=slice("1941","2003"))

In [ ]:
coslat = np.cos(np.deg2rad(era5_slp_anoms1951_eap.coords['latitude'].values))
wgts = np.sqrt(coslat)[..., np.newaxis]
# read a spatial-temporal field, time must be the first dimension
#eofs_ekf400_monthly = iris.load_cube("/Volumes/SPARK/ekf400v2/ensmean/EKF400_ensmean_v2.0_JJA_v200_anoms1951_CGTdomain.nc")
# create a solver class, taking advantage of built-in weighting
eofs_monthly  = Eof(era5_slp_anoms1951_eap , weights=wgts)
variance_fractions_monthly = eofs_monthly.varianceFraction(neigs=3)
pcs = eofs_monthly.pcs(npcs=3, pcscaling=1)

# retrieve the first two EOFs from the solver class
eofs_monthly  = eofs_monthly.eofs(neofs=3)
#xr_eofs_ekf400_monthly =xr.DataArray.from_iris(eofs_ekf400_monthly)


In [ ]:
fig = plt.figure(figsize=(10, 5), dpi= 300)
ax = plt.axes(projection=ccrs.EqualEarth(central_longitude=0, globe=None)) 
tplot=eofs_monthly[0,:,:].plot.pcolormesh(ax=ax,
levels = 17, transform=ccrs.PlateCarree(), cmap=current_cmap, cbar_kwargs={'orientation':'horizontal',
'fraction':0.042, 'pad':0.015, 'aspect':35})

tplot.colorbar.set_label('eigenvalues', size=10) 
tplot.ylabel_style = {'size':16}

ax.set_global()
ax.coastlines()
ax.set_title('JJA anoms SLP First EOF ERA5 ensmean wrt 1951-1980')
ax.set_extent([-95, 95, 20, 80])
;
fig.savefig(plot_folder+"JJAslp_1stEOFera19511980_19412003.pdf")

In [ ]:
fig = plt.figure(figsize=(10, 5), dpi= 300)
ax = plt.axes(projection=ccrs.EqualEarth(central_longitude=0, globe=None)) 
tplot=eofs_monthly[1,:,:].plot.pcolormesh(ax=ax,
levels = 17, transform=ccrs.PlateCarree(), cmap=current_cmap, cbar_kwargs={'orientation':'horizontal',
'fraction':0.042, 'pad':0.015, 'aspect':35})

tplot.colorbar.set_label('eigenvalues', size=10) 
tplot.ylabel_style = {'size':16}

ax.set_global()
ax.coastlines()
ax.set_title('JJA anoms SLP Second EOF ERA5 ensmean wrt 1951-1980')
ax.set_extent([-95, 95, 20, 80])
;
fig.savefig(plot_folder+"JJAslp_2ndtEOFera19511980_19412003.pdf")

# CGT EOF

## EKF400v2

In [ ]:

lon_list_CGTdomain_ekf= np.arange(-100, 100+1.875, 1.875)
lat_list_CGTdomain_ekf = np.arange(20, 80+1.875, 1.875)

In [ ]:
ekf=xr.open_dataset(ekf_folder+"EKF400_ensmean_v2.0_remapped.nc")

ekf["time"]=xr.cftime_range(start = '1602-01-01', end = '2003-12-31',freq="M",calendar = 'standard')


In [ ]:
EKF400_ensmean_JJA_v200_anoms1951=create_anomalies(da=ekf.northward_wind.sel(pressure_level_wind=200),month_or_season="JJA",ref_time=["1951-01-01","1980-12-31"])
EKF400_ensmean_JJA_v200_anoms1951_v2=xr.open_dataset("/Volumes/SPARK/ekf400v2/ensmean/EKF400_ensmean_v2.0_JJA_v200_anoms1951.nc")

In [ ]:
EKF400_ensmean_JJA_v200_anoms1901=create_anomalies(da=ekf.northward_wind.sel(pressure_level_wind=200),month_or_season="JJA",ref_time=["1901-01-01","2000-12-31"])


In [ ]:
EKF400_ensmean_JJA_v200_anoms1951_v2_CGT=xr.open_dataset("/Volumes/SPARK/ekf400v2/ensmean/EKF400_ensmean_v2.0_JJA_v200_anoms1951_CGTdomain.nc")

In [ ]:
EKF400_ensmean_JJA_v200_anoms1951_CGT1=EKF400_ensmean_JJA_v200_anoms1951.sel(longitude=lon_list_CGTdomain_ekf,latitude=lat_list_CGTdomain_ekf,method="nearest")

In [ ]:
EKF400_ensmean_JJA_v200_anoms1901_CGT1=EKF400_ensmean_JJA_v200_anoms1901.sel(longitude=lon_list_CGTdomain_ekf,latitude=lat_list_CGTdomain_ekf,method="nearest")

In [ ]:
#solver = Eof(data_array)

In [ ]:
coslat = np.cos(np.deg2rad(EKF400_ensmean_JJA_v200_anoms1951_CGT1.coords['latitude'].values))
wgts = np.sqrt(coslat)[..., np.newaxis]
# read a spatial-temporal field, time must be the first dimension
#eofs_ekf400_monthly = iris.load_cube("/Volumes/SPARK/ekf400v2/ensmean/EKF400_ensmean_v2.0_JJA_v200_anoms1951_CGTdomain.nc")
# create a solver class, taking advantage of built-in weighting
eofs_ekf400_monthly  = Eof(EKF400_ensmean_JJA_v200_anoms1951_CGT1 , weights=wgts)
variance_fractions_ekf400_monthly = eofs_ekf400_monthly.varianceFraction(neigs=3)
pcs_CGT_allyears = eofs_ekf400_monthly.pcs(npcs=3, pcscaling=1)

# retrieve the first two EOFs from the solver class
eofs_ekf400_monthly  = eofs_ekf400_monthly.eofs(neofs=3)
#xr_eofs_ekf400_monthly =xr.DataArray.from_iris(eofs_ekf400_monthly)


In [ ]:
fig = plt.figure(figsize=(10, 5), dpi= 300)
ax = plt.axes(projection=ccrs.EqualEarth(central_longitude=0, globe=None)) 
tplot=eofs_ekf400_monthly[0,:,:].plot.pcolormesh(ax=ax,
levels = 17, transform=ccrs.PlateCarree(), cmap=current_cmap, cbar_kwargs={'orientation':'horizontal',
'fraction':0.042, 'pad':0.015, 'aspect':35})

tplot.colorbar.set_label('eigenvalues', size=10) 
tplot.ylabel_style = {'size':16}

ax.set_global()
ax.coastlines()
ax.set_title('JJA anoms v200 First EOF EKF400 wrt 1951-1980')
ax.set_extent([-95, 95, 20, 80])
;
fig.savefig(plot_folder+"JJAv200_1stEOFEKF19511980.pdf")

In [ ]:
EKF400_ensmean_JJA_v200_anoms1951_CGT1_1941_2003=EKF400_ensmean_JJA_v200_anoms1951_CGT1.sel(time=slice("1941","2003"))

In [ ]:
coslat = np.cos(np.deg2rad(EKF400_ensmean_JJA_v200_anoms1951_CGT1_1941_2003.coords['latitude'].values))
wgts = np.sqrt(coslat)[..., np.newaxis]
# read a spatial-temporal field, time must be the first dimension
#eofs_ekf400_monthly = iris.load_cube("/Volumes/SPARK/ekf400v2/ensmean/EKF400_ensmean_v2.0_JJA_v200_anoms1951_CGTdomain.nc")
# create a solver class, taking advantage of built-in weighting
eofs_ekf400_monthly  = Eof(EKF400_ensmean_JJA_v200_anoms1951_CGT1_1941_2003 , weights=wgts)
variance_fractions_ekf400_monthly = eofs_ekf400_monthly.varianceFraction(neigs=3)
pcs_ekf = eofs_ekf400_monthly.pcs(npcs=3, pcscaling=1)
# retrieve the first two EOFs from the solver class
eofs_ekf400_monthly  = eofs_ekf400_monthly.eofs(neofs=3)
#xr_eofs_ekf400_monthly =xr.DataArray.from_iris(eofs_ekf400_monthly)


In [ ]:
fig = plt.figure(figsize=(10, 5), dpi= 300)
ax = plt.axes(projection=ccrs.EqualEarth(central_longitude=0, globe=None)) 
tplot=eofs_ekf400_monthly[0,:,:].plot.pcolormesh(ax=ax,
levels = 17, transform=ccrs.PlateCarree(), cmap=current_cmap, cbar_kwargs={'orientation':'horizontal',
'fraction':0.042, 'pad':0.015, 'aspect':35})

tplot.colorbar.set_label('eigenvalues', size=10) 
tplot.ylabel_style = {'size':16}

ax.set_global()
ax.coastlines()
ax.set_title('JJA anoms v200 First EOF EKF400 wrt 1951-1980')
ax.set_extent([-95, 95, 20, 80])
;
fig.savefig(plot_folder+"JJAv200_1stEOFEKF19511980_19412003.pdf")

## CERA20C

In [ ]:
lon_list_CGTdomain= np.arange(-100, 100+1, 1)
lat_list_CGTdomain = np.arange(20, 80+1, 1)

In [ ]:
cera20c_uv=xr.open_dataset(cera20c_folder+"cera20c_uv250_monthly.nc")
cera20c_uv_ensmean=cera20c_uv.mean(dim="number")

In [ ]:
cera20c_uv_anoms1951=create_anomalies_alt(da=cera20c_uv_ensmean.v,month_or_season="JJA",ref_time=["1951-01-01","1980-12-31"])


In [ ]:
cera20c_uv_anoms1951_CGT1=cera20c_uv_anoms1951.sel(longitude=lon_list_CGTdomain,latitude=lat_list_CGTdomain,method="nearest")

In [ ]:
coslat = np.cos(np.deg2rad(cera20c_uv_anoms1951_CGT1.coords['latitude'].values))
wgts = np.sqrt(coslat)[..., np.newaxis]
# read a spatial-temporal field, time must be the first dimension
#eofs_ekf400_monthly = iris.load_cube("/Volumes/SPARK/ekf400v2/ensmean/EKF400_ensmean_v2.0_JJA_v200_anoms1951_CGTdomain.nc")
# create a solver class, taking advantage of built-in weighting
eofs_monthly  = Eof(cera20c_uv_anoms1951_CGT1 , weights=wgts)
variance_fractions_monthly = eofs_monthly.varianceFraction(neigs=3)
pcs_cera= eofs_monthly.pcs(npcs=3, pcscaling=1)

# retrieve the first two EOFs from the solver class
eofs_monthly  = eofs_monthly.eofs(neofs=3)
#xr_eofs_ekf400_monthly =xr.DataArray.from_iris(eofs_ekf400_monthly)


In [ ]:
fig = plt.figure(figsize=(10, 5), dpi= 300)
ax = plt.axes(projection=ccrs.EqualEarth(central_longitude=0, globe=None)) 
tplot=eofs_monthly[0,:,:].plot.pcolormesh(ax=ax,
levels = 17, transform=ccrs.PlateCarree(), cmap=current_cmap, cbar_kwargs={'orientation':'horizontal',
'fraction':0.042, 'pad':0.015, 'aspect':35})

tplot.colorbar.set_label('eigenvalues', size=10) 
tplot.ylabel_style = {'size':16}

ax.set_global()
ax.coastlines()
ax.set_title('JJA anoms v250 First EOF CERA20C wrt 1951-1980')
ax.set_extent([-95, 95, 20, 80])
;
fig.savefig(plot_folder+"JJAv200_1stEOFcera20c19511980.pdf")

In [ ]:
cera20c_uv_anoms1951_CGT1_1941_2003=cera20c_uv_anoms1951_CGT1.sel(time=slice("1941","2003"))

In [ ]:
coslat = np.cos(np.deg2rad(cera20c_uv_anoms1951_CGT1_1941_2003.coords['latitude'].values))
wgts = np.sqrt(coslat)[..., np.newaxis]
# read a spatial-temporal field, time must be the first dimension
#eofs_ekf400_monthly = iris.load_cube("/Volumes/SPARK/ekf400v2/ensmean/EKF400_ensmean_v2.0_JJA_v200_anoms1951_CGTdomain.nc")
# create a solver class, taking advantage of built-in weighting
eofs_monthly  = Eof(cera20c_uv_anoms1951_CGT1_1941_2003 , weights=wgts)
variance_fractions_monthly = eofs_monthly.varianceFraction(neigs=3)
pcs_cera= eofs_monthly.pcs(npcs=3, pcscaling=1)

# retrieve the first two EOFs from the solver class
eofs_monthly  = eofs_monthly.eofs(neofs=3)
#xr_eofs_ekf400_monthly =xr.DataArray.from_iris(eofs_ekf400_monthly)


In [ ]:
fig = plt.figure(figsize=(10, 5), dpi= 300)
ax = plt.axes(projection=ccrs.EqualEarth(central_longitude=0, globe=None)) 
tplot=eofs_monthly[0,:,:].plot.pcolormesh(ax=ax,
levels = 17, transform=ccrs.PlateCarree(), cmap=current_cmap, cbar_kwargs={'orientation':'horizontal',
'fraction':0.042, 'pad':0.015, 'aspect':35})

tplot.colorbar.set_label('eigenvalues', size=10) 
tplot.ylabel_style = {'size':16}

ax.set_global()
ax.coastlines()
ax.set_title('JJA anoms v250 First EOF CERA20C wrt 1951-1980')
ax.set_extent([-95, 95, 20, 80])
;
fig.savefig(plot_folder+"JJAv200_1stEOFcera20c19511980_19412003.pdf")

## 20crv3

In [ ]:
lon_list_CGTdomain= np.arange(-100, 100+1, 1)
lat_list_CGTdomain = np.arange(20, 80+1, 1)

In [ ]:
tcrv3_v=xr.open_dataset(tcr_folder+"vwnd.mon.mean_remapped.nc")
tcrv3_v=tcrv3_v.sel(level=250)

In [ ]:
tcrv3_v_anoms1951=create_anomalies(da=tcrv3_v.vwnd,month_or_season="JJA",ref_time=["1951-01-01","1980-12-31"])


In [ ]:
tcrv3_v_anoms1951_CGT1=tcrv3_v_anoms1951.sel(lon=lon_list_CGTdomain,lat=lat_list_CGTdomain,method="nearest")

In [ ]:
coslat = np.cos(np.deg2rad(tcrv3_v_anoms1951_CGT1.coords['lat'].values))
wgts = np.sqrt(coslat)[..., np.newaxis]
# read a spatial-temporal field, time must be the first dimension
#eofs_ekf400_monthly = iris.load_cube("/Volumes/SPARK/ekf400v2/ensmean/EKF400_ensmean_v2.0_JJA_v200_anoms1951_CGTdomain.nc")
# create a solver class, taking advantage of built-in weighting
eofs_monthly  = Eof(tcrv3_v_anoms1951_CGT1 , weights=wgts)
variance_fractions_monthly = eofs_monthly.varianceFraction(neigs=3)
pcs_20cr = eofs_monthly.pcs(npcs=3, pcscaling=1)

# retrieve the first two EOFs from the solver class
eofs_monthly  = eofs_monthly.eofs(neofs=3)
#xr_eofs_ekf400_monthly =xr.DataArray.from_iris(eofs_ekf400_monthly)


In [ ]:
fig = plt.figure(figsize=(10, 5), dpi= 300)
ax = plt.axes(projection=ccrs.EqualEarth(central_longitude=0, globe=None)) 
tplot=eofs_monthly[0,:,:].plot.pcolormesh(ax=ax,
levels = 17, transform=ccrs.PlateCarree(), cmap=current_cmap, cbar_kwargs={'orientation':'horizontal',
'fraction':0.042, 'pad':0.015, 'aspect':35})

tplot.colorbar.set_label('eigenvalues', size=10) 
tplot.ylabel_style = {'size':16}

ax.set_global()
ax.coastlines()
ax.set_title('JJA anoms v250 First EOF 20CRv3 wrt 1951-1980')
ax.set_extent([-95, 95, 20, 80])
;
fig.savefig(plot_folder+"JJAv200_1stEOF20CRv319511980.pdf")

In [ ]:
tcrv3_v_anoms1951_CGT1_1941_2003=tcrv3_v_anoms1951_CGT1.sel(time=slice("1941","2003"))

In [ ]:
coslat = np.cos(np.deg2rad(tcrv3_v_anoms1951_CGT1_1941_2003.coords['lat'].values))
wgts = np.sqrt(coslat)[..., np.newaxis]
# read a spatial-temporal field, time must be the first dimension
#eofs_ekf400_monthly = iris.load_cube("/Volumes/SPARK/ekf400v2/ensmean/EKF400_ensmean_v2.0_JJA_v200_anoms1951_CGTdomain.nc")
# create a solver class, taking advantage of built-in weighting
eofs_monthly  = Eof(tcrv3_v_anoms1951_CGT1_1941_2003 , weights=wgts)
variance_fractions_monthly = eofs_monthly.varianceFraction(neigs=3)
pcs_20cr = eofs_monthly.pcs(npcs=3, pcscaling=1)

# retrieve the first two EOFs from the solver class
eofs_monthly  = eofs_monthly.eofs(neofs=3)
#xr_eofs_ekf400_monthly =xr.DataArray.from_iris(eofs_ekf400_monthly)


In [ ]:
fig = plt.figure(figsize=(10, 5), dpi= 300)
ax = plt.axes(projection=ccrs.EqualEarth(central_longitude=0, globe=None)) 
tplot=eofs_monthly[0,:,:].plot.pcolormesh(ax=ax,
levels = 17, transform=ccrs.PlateCarree(), cmap=current_cmap, cbar_kwargs={'orientation':'horizontal',
'fraction':0.042, 'pad':0.015, 'aspect':35})

tplot.colorbar.set_label('eigenvalues', size=10) 
tplot.ylabel_style = {'size':16}

ax.set_global()
ax.coastlines()
ax.set_title('JJA anoms v250 First EOF 20CRv3 wrt 1951-1980')
ax.set_extent([-95, 95, 20, 80])
;
fig.savefig(plot_folder+"JJAv200_1stEOF20CRv319511980_19412003.pdf")

## ERA5

In [ ]:
lon_list_CGTdomain= np.arange(-100, 100+0.25, 0.25)
lat_list_CGTdomain = np.arange(20, 80+0.25, 0.25)

In [ ]:
era5_v200=xr.open_dataset(era5_folder+"era5_v200_remapped.nc")

In [ ]:
era5_v200_regrid=era5_v200.regrid.linear(ekf)

In [ ]:
era5_v200_anoms1951=create_anomalies(da=era5_v200.v,month_or_season="JJA",ref_time=["1951-01-01","1980-12-31"])


In [ ]:
era5_v200_regrid_anoms1951=create_anomalies(da=era5_v200_regrid.v,month_or_season="JJA",ref_time=["1951-01-01","1980-12-31"])


In [ ]:
era5_v200_anoms1951_CGT1=era5_v200_anoms1951.sel(longitude=lon_list_CGTdomain,latitude=lat_list_CGTdomain,method="nearest")

In [ ]:
era5_v200_anoms1951_CGT1_1941_2003=era5_v200_anoms1951_CGT1.sel(time=slice("1941","2003"))

In [ ]:
era5_v200_regrid_anoms1951_CGT1=era5_v200_regrid_anoms1951.sel(longitude=lon_list_CGTdomain_ekf,latitude=lat_list_CGTdomain_ekf,method="nearest")

In [ ]:
era5_v200_regrid_anoms1951_CGT1_1941_2003=era5_v200_regrid_anoms1951_CGT1.sel(time=slice("1941","2003"))

In [ ]:
coslat = np.cos(np.deg2rad(era5_v200_anoms1951_CGT1.coords['latitude'].values))
wgts = np.sqrt(coslat)[..., np.newaxis]
# read a spatial-temporal field, time must be the first dimension
#eofs_ekf400_monthly = iris.load_cube("/Volumes/SPARK/ekf400v2/ensmean/EKF400_ensmean_v2.0_JJA_v200_anoms1951_CGTdomain.nc")
# create a solver class, taking advantage of built-in weighting
eofs_monthly  = Eof(era5_v200_anoms1951_CGT1 , weights=wgts)
variance_fractions_monthly = eofs_monthly.varianceFraction(neigs=3)
pcs_era5 = eofs_monthly.pcs(npcs=3, pcscaling=1)

# retrieve the first two EOFs from the solver class
eofs_monthly  = eofs_monthly.eofs(neofs=3)
#xr_eofs_ekf400_monthly =xr.DataArray.from_iris(eofs_ekf400_monthly)


In [ ]:
fig = plt.figure(figsize=(10, 5), dpi= 300)
ax = plt.axes(projection=ccrs.EqualEarth(central_longitude=0, globe=None)) 
tplot=eofs_monthly[0,:,:].plot.pcolormesh(ax=ax,
levels = 17, transform=ccrs.PlateCarree(), cmap=current_cmap, cbar_kwargs={'orientation':'horizontal',
'fraction':0.042, 'pad':0.015, 'aspect':35})

tplot.colorbar.set_label('eigenvalues', size=10) 
tplot.ylabel_style = {'size':16}

ax.set_global()
ax.coastlines()
ax.set_title('JJA anoms v200 First EOF ERA5 wrt 1951-1980')
ax.set_extent([-95, 95, 20, 80])
;
fig.savefig(plot_folder+"JJAv200_1stEOFERA519511980.pdf")

In [ ]:
coslat = np.cos(np.deg2rad(era5_v200_anoms1951_CGT1_1941_2003.coords['latitude'].values))
wgts = np.sqrt(coslat)[..., np.newaxis]
# read a spatial-temporal field, time must be the first dimension
#eofs_ekf400_monthly = iris.load_cube("/Volumes/SPARK/ekf400v2/ensmean/EKF400_ensmean_v2.0_JJA_v200_anoms1951_CGTdomain.nc")
# create a solver class, taking advantage of built-in weighting
eofs_monthly  = Eof(era5_v200_anoms1951_CGT1_1941_2003 , weights=wgts)
variance_fractions_monthly = eofs_monthly.varianceFraction(neigs=3)
pcs_era5 = eofs_monthly.pcs(npcs=3, pcscaling=1)

# retrieve the first two EOFs from the solver class
eofs_monthly  = eofs_monthly.eofs(neofs=3)
#xr_eofs_ekf400_monthly =xr.DataArray.from_iris(eofs_ekf400_monthly)


In [ ]:
fig = plt.figure(figsize=(10, 5), dpi= 300)
ax = plt.axes(projection=ccrs.EqualEarth(central_longitude=0, globe=None)) 
tplot=eofs_monthly[0,:,:].plot.pcolormesh(ax=ax,
levels = 17, transform=ccrs.PlateCarree(), cmap=current_cmap, cbar_kwargs={'orientation':'horizontal',
'fraction':0.042, 'pad':0.015, 'aspect':35})

tplot.colorbar.set_label('eigenvalues', size=10) 
tplot.ylabel_style = {'size':16}

ax.set_global()
ax.coastlines()
ax.set_title('JJA anoms v200 First EOF ERA5 wrt 1951-1980')
ax.set_extent([-95, 95, 20, 80])
;
fig.savefig(plot_folder+"JJAv200_1stEOFERA519511980_19412003.pdf")

## ERA5 vs EKF400v2 comparison

In [ ]:
pcs_ekf["time"]=pcs_era5.time

In [ ]:
pcs_20cr["time"]=pcs_era5.time

In [ ]:
pcs_cera["time"]=pcs_era5.time

In [ ]:
pcs_era5=pcs_era5*-1

In [ ]:
plt.figure(figsize=(10, 5), dpi= 300)

pcs_era5.sel(mode=0).plot(label="ERA5")
pcs_ekf.sel(mode=0).plot(label="EKF400v2 ensmean")
pcs_20cr.sel(mode=0).plot(label="20CRv3 ensmean")
pcs_cera.sel(mode=0).plot(label="CERA20C ensmean")
plt.title("1st Principal Component JJA V200 anomalies CGT Domain")
plt.legend()
plt.savefig(plot_folder+"1stPCeraekf_v200_cgt.pdf")

In [ ]:
era5_ekf400_v200_corr=xr.corr(era5_v200_regrid_anoms1951_CGT1_1941_2003,EKF400_ensmean_JJA_v200_anoms1951_CGT1_1941_2003,dim="time")

In [ ]:
fig = plt.figure(figsize=(10, 5), dpi= 300)
ax = plt.axes(projection=ccrs.EqualEarth(central_longitude=0, globe=None)) 
tplot=era5_ekf400_v200_corr.plot.pcolormesh(ax=ax,
levels = 17, transform=ccrs.PlateCarree(), cmap=current_cmap,vmax=1,vmin=-1, cbar_kwargs={'orientation':'horizontal',
'fraction':0.042, 'pad':0.015, 'aspect':35})

tplot.colorbar.set_label('[Corr. Coeff.]', size=10) 
tplot.ylabel_style = {'size':16}

ax.set_global()
ax.coastlines()
ax.set_title('JJA anoms v200 corr ERA5 vs. EKF400v2 ensmean 1941-2003')
ax.set_extent([-95, 95, 20, 80])
;
fig.savefig(plot_folder+"era5_ekf400_v200_corr.pdf")

# read in all gridded data, calculate JJA mean

In [ ]:
ekf=xr.open_dataset(ekf_folder+"EKF400_ensmean_v2.0_remapped.nc")

ekf["time"]=xr.cftime_range(start = '1602-01-01', end = '2003-12-31',freq="M",calendar = 'standard')


In [ ]:
ekf=xr.merge([ekf['air_temperature'],ekf['total precipitation'],ekf['air_pressure_at_sea_level'],ekf['eastward_wind'].sel(pressure_level_wind=200),ekf['northward_wind'].sel(pressure_level_wind=200),ekf['lagrangian_tendency_of_air_pressure'],ekf['blocks'],ekf['cycfreq']])

In [ ]:
ekf_JJA_anoms1901=create_anomalies_alt(da=ekf,month_or_season="JJA",ref_time=["1901-01-01","2000-12-31"])
ekf_JJA_anoms1901_1836_2003=ekf_JJA_anoms1901.sel(time=slice("1836","2003"))
ekf_JJA_anoms1901_1901_2003=ekf_JJA_anoms1901.sel(time=slice("1901","2003"))


In [ ]:
# winter
ekf_jan_anoms1901=create_anomalies_alt(da=ekf,month_or_season=1,ref_time=["1901-01-01","2000-12-31"])
ekf_feb_anoms1901=create_anomalies_alt(da=ekf,month_or_season=2,ref_time=["1901-01-01","2000-12-31"])
ekf_mar_anoms1901=create_anomalies_alt(da=ekf,month_or_season=3,ref_time=["1901-01-01","2000-12-31"])
ekf_apr_anoms1901=create_anomalies_alt(da=ekf,month_or_season=4,ref_time=["1901-01-01","2000-12-31"])
ekf_may_anoms1901=create_anomalies_alt(da=ekf,month_or_season=5,ref_time=["1901-01-01","2000-12-31"])
ekf_jun_anoms1901=create_anomalies_alt(da=ekf,month_or_season=6,ref_time=["1901-01-01","2000-12-31"])
ekf_jul_anoms1901=create_anomalies_alt(da=ekf,month_or_season=7,ref_time=["1901-01-01","2000-12-31"])
ekf_aug_anoms1901=create_anomalies_alt(da=ekf,month_or_season=8,ref_time=["1901-01-01","2000-12-31"])


In [ ]:
single_months_ekf_slp_anoms1901=[ekf_jan_anoms1901['air_pressure_at_sea_level'],ekf_feb_anoms1901['air_pressure_at_sea_level'],ekf_mar_anoms1901['air_pressure_at_sea_level'],ekf_apr_anoms1901['air_pressure_at_sea_level'],ekf_may_anoms1901['air_pressure_at_sea_level'],ekf_jun_anoms1901['air_pressure_at_sea_level'],ekf_jul_anoms1901['air_pressure_at_sea_level'],ekf_aug_anoms1901['air_pressure_at_sea_level']]

In [ ]:
single_months_ekf_v250_anoms1901=[ekf_jan_anoms1901['northward_wind'],ekf_feb_anoms1901['northward_wind'],ekf_mar_anoms1901['northward_wind'],ekf_apr_anoms1901['northward_wind'],ekf_may_anoms1901['northward_wind'],ekf_jun_anoms1901['northward_wind'],ekf_jul_anoms1901['northward_wind'],ekf_aug_anoms1901['northward_wind']]

In [ ]:
tcrv3_t2m=xr.open_dataset(tcr_folder+"air.2m.mon.mean_remapped.nc")
tcrv3_slp=xr.open_dataset(tcr_folder+"prmsl.mon.mean_remapped.nc")
tcrv3_prate=xr.open_dataset(tcr_folder+"prate.mon.mean_remapped.nc")
tcrv3_u=xr.open_dataset(tcr_folder+"uwnd.mon.mean_remapped.nc")
tcrv3_u=tcrv3_u.sel(level=250)
tcrv3_v=xr.open_dataset(tcr_folder+"vwnd.mon.mean_remapped.nc")
tcrv3_v=tcrv3_v.sel(level=250)

In [ ]:
tcrv3=xr.merge([tcrv3_t2m,tcrv3_slp,tcrv3_prate,tcrv3_u,tcrv3_v])

In [ ]:
tcrv3_JJA_anoms1901=create_anomalies_alt(da=tcrv3.drop('level', dim=None),month_or_season="JJA",ref_time=["1901-01-01","2000-12-31"])
tcrv3_JJA_anoms1901_1836_2003=tcrv3_JJA_anoms1901.sel(time=slice("1836","2003"))
tcrv3_JJA_anoms1901_1836_2008=tcrv3_JJA_anoms1901.sel(time=slice("1836","2008"))
tcrv3_JJA_anoms1901_1901_2010=tcrv3_JJA_anoms1901.sel(time=slice("1901","2010"))
tcrv3_JJA_anoms1901_1836_2013=tcrv3_JJA_anoms1901.sel(time=slice("1836","2013"))

                                                    

In [ ]:
cera20c_uv=xr.open_dataset(cera20c_folder+"cera20c_uv250_monthly.nc")
cera20c_uv_ensmean=cera20c_uv.mean(dim="number")

In [ ]:
cera20c_t2mslp=xr.open_dataset(cera20c_folder+"cera20c_t2mslp_monthly.nc")
cera20c_t2mslp_ensmean=cera20c_t2mslp.mean(dim="number")

In [ ]:
cera20c_precip=xr.open_dataset(cera20c_folder+"cera20c_precip_monthly.nc")
cera20c_precip_ensmean=cera20c_precip.mean(dim="number")

In [ ]:
cera20c=xr.merge([cera20c_uv_ensmean,cera20c_t2mslp_ensmean,cera20c_precip_ensmean])

In [ ]:
cera20c_JJA_anoms1901=create_anomalies_alt(da=cera20c,month_or_season="JJA",ref_time=["1901-01-01","2000-12-31"])
cera20c_JJA_anoms1901_1901_2003=cera20c_JJA_anoms1901.sel(time=slice("1901","2003"))
cera20c_JJA_anoms1901_1901_2010=cera20c_JJA_anoms1901.sel(time=slice("1901","2010"))
cera20c_JJA_anoms1901_1901_2008=cera20c_JJA_anoms1901.sel(time=slice("1901","2008"))

In [ ]:
modera_slp=xr.open_dataset(paleora_folder+"ModE-RA_ensmean_slp_anom_wrt_1901-2000_1421-2008_mon_remapped.nc")
modera_t2m=xr.open_dataset(paleora_folder+"ModE-RA_ensmean_temp2_anom_wrt_1901-2000_1421-2008_mon_remapped.nc")
modera_totprec=xr.open_dataset(paleora_folder+"ModE-RA_ensmean_totprec_anom_wrt_1901-2000_1421-2008_mon_remapped.nc")

In [ ]:
#modera_slp=xr.open_mfdataset(paleora_folder+"*_slp_anom_wrt_1901-2000_1421-2008_mon.nc", combine='nested',concat_dim="member")
#modera_slp_ensmean=modera_slp.mean(dim="member")

In [ ]:
#modera_totprec=xr.open_mfdataset(paleora_folder+"*_totprec_anom_wrt_1901-2000_1421-2008_mon.nc", combine='nested',concat_dim="member")
#modera_totprec_ensmean=modera_totprec.mean(dim="member")

In [ ]:
modera=xr.merge([modera_t2m,modera_slp,modera_totprec])
#ds_seasons=modera.resample(time="QS-DEC").mean()
#JJA=ds_seasons.time.dt.month.isin([6])
#modera_JJA_anoms1901=ds_seasons.where(JJA,drop=True)

In [ ]:
modera_JJA_anoms1901=create_season_resolution(da=modera,month_or_season="JJA")


In [ ]:
modera_JJA_anoms1901_1836_2008=modera_JJA_anoms1901.sel(time=slice("1836","2008"))
modera_JJA_anoms1901_1901_2008=modera_JJA_anoms1901.sel(time=slice("1901","2008"))
modera_JJA_anoms1901_1602_2003=modera_JJA_anoms1901.sel(time=slice("1602","2003"))
modera_JJA_anoms1901_1600_2008=modera_JJA_anoms1901.sel(time=slice("1600","2008"))

                                              

In [ ]:
# winter
modera_jan_anoms1901=create_season_resolution(da=modera,month_or_season=1)
modera_feb_anoms1901=create_season_resolution(da=modera,month_or_season=2)
modera_mar_anoms1901=create_season_resolution(da=modera,month_or_season=3)
modera_apr_anoms1901=create_season_resolution(da=modera,month_or_season=4)
modera_may_anoms1901=create_season_resolution(da=modera,month_or_season=5)
modera_jun_anoms1901=create_season_resolution(da=modera,month_or_season=6)
modera_jul_anoms1901=create_season_resolution(da=modera,month_or_season=7)
modera_aug_anoms1901=create_season_resolution(da=modera,month_or_season=8)



# Excursion: Fig 1

In [ ]:
ekf_t2m_JJA2003_anoms=ekf_JJA_anoms1901.air_temperature.sel(time=slice("2003-06-01","2003-09-01"))

In [ ]:
modera_t2m_JJA2003_anoms=modera_JJA_anoms1901.temp2.sel(time=slice("2003-06-01","2003-09-01"))

In [ ]:

fig = plt.figure(figsize=(8, 4), dpi= 200)
ax = plt.axes(projection=ccrs.EqualEarth(central_longitude=10, globe=None)) 
tplot=modera_t2m_JJA2003_anoms.plot.pcolormesh(ax=ax,vmax=5,vmin=-5,
levels = 17, transform=ccrs.PlateCarree(), cmap="coolwarm", cbar_kwargs={'orientation':'vertical',
'fraction':0.012, 'pad':0.015, 'aspect':35})
ax.scatter(lons_new,lats_new,transform=ccrs.PlateCarree(),s=40,marker="d",facecolor='yellow',edgecolor="k",zorder=2)

#ax.scatter(lons[25:29],lats[25:29],transform=ccrs.PlateCarree(),s=50,marker='*',facecolor='yellow')



tplot.colorbar.set_label('[K]', size=10) 
tplot.ylabel_style = {'size':16}

ax.set_global()
ax.coastlines()
ax.set_title('2003 JJA Heatwave in ModE-RA wrt to 1901-2000')
ax.set_extent([-20, 40, 30, 80])
;
fig.savefig(plot_folder+"2003_tree_locations_modera.pdf")

In [ ]:

fig = plt.figure(figsize=(8, 4), dpi= 200)
ax = plt.axes(projection=ccrs.EqualEarth(central_longitude=10, globe=None)) 
tplot=ekf_t2m_JJA2003_anoms.plot.pcolormesh(ax=ax,vmax=5,vmin=-5,
levels = 17, transform=ccrs.PlateCarree(), cmap="coolwarm", cbar_kwargs={'orientation':'vertical',
'fraction':0.012, 'pad':0.015, 'aspect':35})
ax.scatter(lons_new,lats_new,transform=ccrs.PlateCarree(),s=40,marker="d",facecolor='yellow',edgecolor="k",zorder=2)

#ax.scatter(lons[25:29],lats[25:29],transform=ccrs.PlateCarree(),s=50,marker='*',facecolor='yellow')



tplot.colorbar.set_label('[K]', size=10) 
tplot.ylabel_style = {'size':16}

ax.set_global()
ax.coastlines()
ax.set_title('2003 JJA Heatwave in EKF400v2 wrt to 1901-2000')
ax.set_extent([-20, 40, 30, 80])
;
fig.savefig(plot_folder+"2003_tree_locations_ekf.pdf")

# enlarge tree pc1 to grids

In [ ]:
enlarged_ekf=enlarge_grid(pc1df_ekf.values,ekf_JJA_anoms1901.air_temperature,"PC1 o18 new")

In [ ]:
enlarged_tcr=enlarge_grid(pc1df_tcr.values,tcrv3_JJA_anoms1901_1836_2013.prmsl,"PC1 o18 new",lon_name="lon",lat_name="lat")

In [ ]:
enlarged_cera20c=enlarge_grid(pc1df_cera20c.values,cera20c_JJA_anoms1901.t2m,"PC1 o18 new")

In [ ]:
enlarged_modera=enlarge_grid(pc1df_modera.values,modera_JJA_anoms1901_1600_2008.temp2,"PC1 o18 new")

## detrend

In [ ]:
enlarged_ekf_detrend=detrend_dim(enlarged_ekf, dim="time", deg=1)

In [ ]:
enlarged_tcr_detrend=detrend_dim(enlarged_tcr, dim="time", deg=1)

In [ ]:
enlarged_cera20c_detrend=detrend_dim(enlarged_cera20c, dim="time", deg=1)

In [ ]:
enlarged_modera_detrend=detrend_dim(enlarged_modera, dim="time", deg=1)

## correlation between cgt pcs and surface variables

In [ ]:
enlarged_pcs_CGT_allyears_ekf=enlarge_grid(pcs_CGT_allyears.sel(mode=0).values,ekf_JJA_anoms1901.air_temperature,"PC1 CGT")

In [ ]:
enlarged_pcs_CGT_allyears_era=enlarge_grid(pcs_era5.sel(mode=0).values,era5_slp_anoms1951_eap,"PC1 CGT")

In [ ]:
xr_R,xr_R_masked,xr_p=corr_with_sig(enlarged_pcs_CGT_allyears_era,detrend_dim(era5_slp_anoms1951_eap, dim="time", deg=1),sig_level=0.05)

In [ ]:
fig = plt.figure(figsize=(8, 4), dpi= 200)
ax = plt.axes(projection=ccrs.EqualEarth(central_longitude=0, globe=None)) 
tplot=xr_R_masked.plot.pcolormesh(ax=ax,
levels = 17, transform=ccrs.PlateCarree(), vmax=0.6, cmap=current_cmap, cbar_kwargs={'orientation':'vertical',
'fraction':0.012, 'pad':0.015, 'aspect':35})
#ax.scatter(lons[0:25],lats[0:25],transform=ccrs.PlateCarree(),s=60,marker="D",facecolor='yellow',edgecolor="k",zorder=2)

#ax.scatter(lons[25:29],lats[25:29],transform=ccrs.PlateCarree(),s=50,marker='*',facecolor='yellow')



tplot.colorbar.set_label('Pearson Corr. Coef.', size=10) 
tplot.ylabel_style = {'size':16}

ax.set_global()
ax.coastlines()
ax.set_title("ERA5 SLP")
ax.set_extent([-90, 40, 25, 80])
;
fig.savefig(plot_folder+"CGT_JJA_CORR_ERA5_slp.pdf")

In [ ]:
ekf_vars=['air_temperature',
 'total precipitation',
 'air_pressure_at_sea_level',
 'lagrangian_tendency_of_air_pressure',
 'blocks',
 'cycfreq']

for var in ekf_vars:
    xr_R,xr_R_masked,xr_p=corr_with_sig(detrend_dim(enlarged_pcs_CGT_allyears_ekf, dim="time", deg=1).sel(time=slice("1941","2003")),detrend_dim(ekf_JJA_anoms1901[var], dim="time", deg=1).sel(time=slice("1941","2003")),sig_level=0.05)
    fig = plt.figure(figsize=(8, 4), dpi= 200)
    ax = plt.axes(projection=ccrs.EqualEarth(central_longitude=0, globe=None)) 
    tplot=xr_R_masked.plot.pcolormesh(ax=ax,
    levels = 17, transform=ccrs.PlateCarree(), vmax=0.6, cmap=current_cmap, cbar_kwargs={'orientation':'vertical',
    'fraction':0.012, 'pad':0.015, 'aspect':35})
    #ax.scatter(lons[0:25],lats[0:25],transform=ccrs.PlateCarree(),s=60,marker="D",facecolor='yellow',edgecolor="k",zorder=2)
    
    #ax.scatter(lons[25:29],lats[25:29],transform=ccrs.PlateCarree(),s=50,marker='*',facecolor='yellow')
    
    
    
    tplot.colorbar.set_label('Pearson Corr. Coef.', size=10) 
    tplot.ylabel_style = {'size':16}
    
    ax.set_global()
    ax.coastlines()
    ax.set_title(var)
    ax.set_extent([-90, 40, 25, 80])
    ;
    fig.savefig(plot_folder+"CGT_JJA_CORR_EKF_"+var+"_19412003.pdf")

In [ ]:
ekf_vars=['air_temperature',
 'total precipitation',
 'air_pressure_at_sea_level',
 'lagrangian_tendency_of_air_pressure',
 'blocks',
 'cycfreq']

for var in ekf_vars:
    xr_R,xr_R_masked,xr_p=corr_with_sig(detrend_dim(enlarged_pcs_CGT_allyears_ekf, dim="time", deg=1).sel(time=slice("1602","2003")),detrend_dim(ekf_JJA_anoms1901[var], dim="time", deg=1).sel(time=slice("1602","2003")),sig_level=0.05)
    fig = plt.figure(figsize=(8, 4), dpi= 200)
    ax = plt.axes(projection=ccrs.EqualEarth(central_longitude=0, globe=None)) 
    tplot=xr_R_masked.plot.pcolormesh(ax=ax,
    levels = 17, transform=ccrs.PlateCarree(), vmax=0.6, cmap=current_cmap, cbar_kwargs={'orientation':'vertical',
    'fraction':0.012, 'pad':0.015, 'aspect':35})
    #ax.scatter(lons[0:25],lats[0:25],transform=ccrs.PlateCarree(),s=60,marker="D",facecolor='yellow',edgecolor="k",zorder=2)
    
    #ax.scatter(lons[25:29],lats[25:29],transform=ccrs.PlateCarree(),s=50,marker='*',facecolor='yellow')
    
    
    
    tplot.colorbar.set_label('Pearson Corr. Coef.', size=10) 
    tplot.ylabel_style = {'size':16}
    
    ax.set_global()
    ax.coastlines()
    ax.set_title(var)
    ax.set_extent([-90, 40, 25, 80])
    ;
    fig.savefig(plot_folder+"CGT_JJA_CORR_EKF_"+var+"_16022003.pdf")

# no time lag Correlations with atmospheric fields

## EKF400v2

In [ ]:
#EKF vs modera 1602-2003
#EKF vs 20CR 1836-2003
#EKF vs cera20c 1901-2003

In [ ]:
#modera vs 20CR 1836-2008
#modera vs cera20 1901-2008

In [ ]:
startyear="1602"
endyear="2003"
vars=list(ekf_JJA_anoms1901.keys())
print(vars)
ekf_JJA_corrs_masked=[]

for var in vars:

    x=detrend_dim(enlarged_ekf, dim="time", deg=1).sel(time=slice(startyear,endyear))
    y=detrend_dim(ekf_JJA_anoms1901[var], dim="time", deg=1).sel(time=slice(startyear,endyear))
    xr_R,xr_R_masked,xr_p=corr_with_sig(x,y,sig_level=0.05)
    ekf_JJA_corrs_masked.append(xr_R_masked)
    
    fig = plt.figure(figsize=(8, 4), dpi= 200)
    ax = plt.axes(projection=ccrs.EqualEarth(central_longitude=0, globe=None)) 
    tplot=xr_R_masked.plot.pcolormesh(ax=ax,
    levels = 17, transform=ccrs.PlateCarree(), vmax=0.6, cmap=current_cmap, cbar_kwargs={'orientation':'vertical',
    'fraction':0.012, 'pad':0.015, 'aspect':35})

    tplot.colorbar.set_label('Pearson Corr. Coef.', size=10) 
    tplot.ylabel_style = {'size':16}
    
    ax.set_global()
    ax.coastlines()
    ax.set_title(var+"_"+startyear+endyear)
    ax.set_extent([-90, 40, 25, 80])
    ;
    fig.savefig(plot_folder+"o18pc1_JJA_CORR_EKF_"+var+"_"+startyear+endyear+".pdf")

In [ ]:
startyear="1602"
endyear="2003"
vars=list(ekf_jan_anoms1901.keys())
print(vars)
ekf_jan_corrs_masked=[]

for var in vars:

    x=detrend_dim(enlarged_ekf, dim="time", deg=1).sel(time=slice(startyear,endyear))
    y=detrend_dim(ekf_jan_anoms1901[var], dim="time", deg=1).sel(time=slice(startyear,endyear))
    xr_R,xr_R_masked,xr_p=corr_with_sig(x,y,sig_level=0.05)
    ekf_jan_corrs_masked.append(xr_R_masked)
    
    fig = plt.figure(figsize=(8, 4), dpi= 200)
    ax = plt.axes(projection=ccrs.EqualEarth(central_longitude=0, globe=None)) 
    tplot=xr_R_masked.plot.pcolormesh(ax=ax,
    levels = 17, transform=ccrs.PlateCarree(), vmax=0.6, cmap=current_cmap, cbar_kwargs={'orientation':'vertical',
    'fraction':0.012, 'pad':0.015, 'aspect':35})

    tplot.colorbar.set_label('Pearson Corr. Coef.', size=10) 
    tplot.ylabel_style = {'size':16}
    
    ax.set_global()
    ax.coastlines()
    ax.set_title(var+"_"+startyear+endyear)
    ax.set_extent([-90, 40, 25, 80])
    ;
    fig.savefig(plot_folder+"o18pc1_jan_CORR_EKF_"+var+"_"+startyear+endyear+".pdf")

In [ ]:
startyear="1602"
endyear="2003"
vars=list(ekf_feb_anoms1901.keys())
print(vars)
ekf_feb_corrs_masked=[]

for var in vars:

    x=detrend_dim(enlarged_ekf, dim="time", deg=1).sel(time=slice(startyear,endyear))
    y=detrend_dim(ekf_feb_anoms1901[var], dim="time", deg=1).sel(time=slice(startyear,endyear))
    xr_R,xr_R_masked,xr_p=corr_with_sig(x,y,sig_level=0.05)
    ekf_feb_corrs_masked.append(xr_R_masked)
    
    fig = plt.figure(figsize=(8, 4), dpi= 200)
    ax = plt.axes(projection=ccrs.EqualEarth(central_longitude=0, globe=None)) 
    tplot=xr_R_masked.plot.pcolormesh(ax=ax,
    levels = 17, transform=ccrs.PlateCarree(), vmax=0.6, cmap=current_cmap, cbar_kwargs={'orientation':'vertical',
    'fraction':0.012, 'pad':0.015, 'aspect':35})

    tplot.colorbar.set_label('Pearson Corr. Coef.', size=10) 
    tplot.ylabel_style = {'size':16}
    
    ax.set_global()
    ax.coastlines()
    ax.set_title(var+"_"+startyear+endyear)
    ax.set_extent([-90, 40, 25, 80])
    ;
    fig.savefig(plot_folder+"o18pc1_feb_CORR_EKF_"+var+"_"+startyear+endyear+".pdf")

In [ ]:
choice=[0,1,2,4,6,7]


ds_merged=xr.concat(ekf_JJA_corrs_masked,dim="vars")
ds_merged=ds_merged.isel(vars=choice)


seasons=list(["t2m","precip","slp","v250","blocks","cyclones"])
INDEX=list(["(a)","(b)","(c)","(d)","(e)","(f)"])
fig, axs = plt.subplots(3, 2, figsize=(10, 7), dpi= 300,subplot_kw={'projection': ccrs.EqualEarth(central_longitude=0, globe=None)})
# axs is a 2 dimensional array of `GeoAxes`.  We will flatten it into a 1-D array
axs=axs.flatten()

#Loop over all of the models
for i in range(6):
    tplot=ds_merged[i,:,:].plot.pcolormesh(ax=axs[i],levels = 17,vmax=0.6, transform=ccrs.PlateCarree(), cmap=current_cmap,add_colorbar=False)
    axs[i].set_title(INDEX[i],loc='left')
    axs[i].coastlines()
    axs[i].set_title(seasons[i])
    axs[i].set_extent([-90, 50, 25, 80])

# Adjust the location of the subplots on the page to make room for the colorbar
fig.subplots_adjust(bottom=0.25, top=0.9, left=0.2, right=0.8,
                    wspace=0.1, hspace=0.01)

# Add a colorbar axis at the bottom of the graph
cbar_ax = fig.add_axes([0.2, 0.2, 0.6, 0.02])

# Draw the colorbar
cbar=fig.colorbar(tplot, cax=cbar_ax,orientation='horizontal')
cbar.set_label('Pearson Corr Coeff', size=10) 




# Make it nice
#plt.tight_layout()
fig.savefig(plot_folder+"o18pc1_JJA_CORR_EKF_"+startyear+endyear+".pdf")

In [ ]:
startyear="1901"
endyear="2003"
vars=list(ekf_JJA_anoms1901.keys())
print(vars)
ekf_JJA_corrs_masked=[]

for var in vars:

    x=detrend_dim(enlarged_ekf, dim="time", deg=1).sel(time=slice(startyear,endyear))
    y=detrend_dim(ekf_JJA_anoms1901[var], dim="time", deg=1).sel(time=slice(startyear,endyear))
    xr_R,xr_R_masked,xr_p=corr_with_sig(x,y,sig_level=0.05)
    ekf_JJA_corrs_masked.append(xr_R_masked)
    
    fig = plt.figure(figsize=(8, 4), dpi= 200)
    ax = plt.axes(projection=ccrs.EqualEarth(central_longitude=0, globe=None)) 
    tplot=xr_R_masked.plot.pcolormesh(ax=ax,
    levels = 17, transform=ccrs.PlateCarree(), vmax=0.6, cmap=current_cmap, cbar_kwargs={'orientation':'vertical',
    'fraction':0.012, 'pad':0.015, 'aspect':35})

    tplot.colorbar.set_label('Pearson Corr. Coef.', size=10) 
    tplot.ylabel_style = {'size':16}
    
    ax.set_global()
    ax.coastlines()
    ax.set_title(var+"_"+startyear+endyear)
    ax.set_extent([-90, 40, 25, 80])
    ;
    fig.savefig(plot_folder+"o18pc1_JJA_CORR_EKF_"+var+"_"+startyear+endyear+".pdf")

In [ ]:
choice=[0,1,2,4,6,7]
choice=[0,1,2,6,4,3]

ds_merged=xr.concat(ekf_JJA_corrs_masked,dim="vars")
ds_merged=ds_merged.isel(vars=choice)


seasons=list(["2m Temp. ","Precip.","SLP","Blocking Freq.","Northward Wind","Eastward Wind"])
INDEX=list(["(a)","(b)","(c)","(d)","(e)","(f)"])
fig, axs = plt.subplots(3, 2, figsize=(10, 7), dpi= 300,subplot_kw={'projection': ccrs.EqualEarth(central_longitude=0, globe=None)})
# axs is a 2 dimensional array of `GeoAxes`.  We will flatten it into a 1-D array
axs=axs.flatten()

#Loop over all of the models
for i in range(6):
    tplot=ds_merged[i,:,:].plot.pcolormesh(ax=axs[i],levels = 17,vmax=0.6, transform=ccrs.PlateCarree(), cmap=current_cmap,add_colorbar=False)
    axs[i].set_title(INDEX[i],loc='left')
    axs[i].coastlines()
    axs[i].set_title(seasons[i])
    axs[i].set_extent([-90, 50, 25, 80])

# Adjust the location of the subplots on the page to make room for the colorbar
fig.subplots_adjust(bottom=0.25, top=0.9, left=0.2, right=0.8,
                    wspace=0.1, hspace=0.01)

# Add a colorbar axis at the bottom of the graph
cbar_ax = fig.add_axes([0.2, 0.2, 0.6, 0.02])

# Draw the colorbar
cbar=fig.colorbar(tplot, cax=cbar_ax,orientation='horizontal')
cbar.set_label('Pearson Corr Coeff', size=10) 




# Make it nice
#plt.tight_layout()
fig.savefig(plot_folder+"o18pc1_JJA_CORR_EKF_"+startyear+endyear+"_uv.pdf")

In [ ]:
choice=[0,1,2,3,6,7]


ds_merged=xr.concat(ekf_jan_corrs_masked,dim="vars")
ds_merged=ds_merged.isel(vars=choice)


seasons=list(["t2m","precip","slp","u250","blocks","cyclones"])
INDEX=list(["(a)","(b)","(c)","(d)","(e)","(f)"])
fig, axs = plt.subplots(3, 2, figsize=(10, 7), dpi= 300,subplot_kw={'projection': ccrs.EqualEarth(central_longitude=0, globe=None)})
# axs is a 2 dimensional array of `GeoAxes`.  We will flatten it into a 1-D array
axs=axs.flatten()

#Loop over all of the models
for i in range(6):
    tplot=ds_merged[i,:,:].plot.pcolormesh(ax=axs[i],levels = 17,vmax=0.6, transform=ccrs.PlateCarree(), cmap=current_cmap,add_colorbar=False)
    axs[i].set_title(INDEX[i],loc='left')
    axs[i].coastlines()
    axs[i].set_title(seasons[i])
    axs[i].set_extent([-90, 50, 25, 80])

# Adjust the location of the subplots on the page to make room for the colorbar
fig.subplots_adjust(bottom=0.25, top=0.9, left=0.2, right=0.8,
                    wspace=0.1, hspace=0.01)

# Add a colorbar axis at the bottom of the graph
cbar_ax = fig.add_axes([0.2, 0.2, 0.6, 0.02])

# Draw the colorbar
cbar=fig.colorbar(tplot, cax=cbar_ax,orientation='horizontal')
cbar.set_label('Pearson Corr Coeff', size=10) 




# Make it nice
#plt.tight_layout()
fig.savefig(plot_folder+"o18pc1_jan_CORR_EKF_"+startyear+endyear+".pdf")

In [ ]:
startyear="1602"
endyear="2003"

ekf_single_corrs_masked=[]

for file in single_months_ekf_slp_anoms1901:

    x=detrend_dim(enlarged_ekf, dim="time", deg=1).sel(time=slice(startyear,endyear))
    y=detrend_dim(file, dim="time", deg=1).sel(time=slice(startyear,endyear))
    xr_R,xr_R_masked,xr_p=corr_with_sig(x,y,sig_level=0.05)
    ekf_single_corrs_masked.append(xr_R_masked)


In [ ]:



ds_merged=xr.concat(ekf_single_corrs_masked,dim="vars")



seasons=list(["jan","feb","mar","apr","may","jun","jul","aug"])
INDEX=list(["(a)","(b)","(c)","(d)","(e)","(f)","(g)","(h)"])
fig, axs = plt.subplots(4, 2, figsize=(10, 10), dpi= 300,subplot_kw={'projection': ccrs.EqualEarth(central_longitude=0, globe=None)})
# axs is a 2 dimensional array of `GeoAxes`.  We will flatten it into a 1-D array
axs=axs.flatten()

#Loop over all of the models
for i in range(8):
    tplot=ds_merged[i,:,:].plot.pcolormesh(ax=axs[i],levels = 17,vmax=0.6, transform=ccrs.PlateCarree(), cmap=current_cmap,add_colorbar=False)
    axs[i].set_title(INDEX[i],loc='left')
    axs[i].coastlines()
    axs[i].set_title(seasons[i])
    axs[i].set_extent([-90, 50, 25, 80])

# Adjust the location of the subplots on the page to make room for the colorbar
fig.subplots_adjust(bottom=0.25, top=0.9, left=0.2, right=0.8,
                    wspace=0.1, hspace=0.1)

# Add a colorbar axis at the bottom of the graph
cbar_ax = fig.add_axes([0.2, 0.2, 0.6, 0.02])

# Draw the colorbar
cbar=fig.colorbar(tplot, cax=cbar_ax,orientation='horizontal')
cbar.set_label('Pearson Corr Coeff', size=10) 




# Make it nice
#plt.tight_layout()
fig.savefig(plot_folder+"o18pc1_months_slp_CORR_EKF_"+startyear+endyear+".pdf")

In [ ]:
startyear="1901"
endyear="2003"

ekf_single_corrs_masked=[]

for file in single_months_ekf_v250_anoms1901:

    x=detrend_dim(enlarged_ekf, dim="time", deg=1).sel(time=slice(startyear,endyear))
    y=detrend_dim(file, dim="time", deg=1).sel(time=slice(startyear,endyear))
    xr_R,xr_R_masked,xr_p=corr_with_sig(x,y,sig_level=0.05)
    ekf_single_corrs_masked.append(xr_R_masked)


In [ ]:



ds_merged=xr.concat(ekf_single_corrs_masked,dim="vars")



seasons=list(["jan","feb","mar","apr","may","jun","jul","aug"])
INDEX=list(["(a)","(b)","(c)","(d)","(e)","(f)","(g)","(h)"])
fig, axs = plt.subplots(4, 2, figsize=(10, 10), dpi= 300,subplot_kw={'projection': ccrs.EqualEarth(central_longitude=0, globe=None)})
# axs is a 2 dimensional array of `GeoAxes`.  We will flatten it into a 1-D array
axs=axs.flatten()

#Loop over all of the models
for i in range(8):
    tplot=ds_merged[i,:,:].plot.pcolormesh(ax=axs[i],levels = 17,vmax=0.6, transform=ccrs.PlateCarree(), cmap=current_cmap,add_colorbar=False)
    axs[i].set_title(INDEX[i],loc='left')
    axs[i].coastlines()
    axs[i].set_title(seasons[i])
    #axs[i].set_extent([-90, 50, 25, 80])

# Adjust the location of the subplots on the page to make room for the colorbar
fig.subplots_adjust(bottom=0.25, top=0.9, left=0.2, right=0.8,
                    wspace=0.1, hspace=0.1)

# Add a colorbar axis at the bottom of the graph
cbar_ax = fig.add_axes([0.2, 0.2, 0.6, 0.02])

# Draw the colorbar
cbar=fig.colorbar(tplot, cax=cbar_ax,orientation='horizontal')
cbar.set_label('Pearson Corr Coeff', size=10) 




# Make it nice
#plt.tight_layout()
fig.savefig(plot_folder+"o18pc1_months_v250_CORR_EKF_"+startyear+endyear+".pdf")

## 20CRv3

In [ ]:
startyear="1836"
endyear="2003"
vars=list(tcrv3_JJA_anoms1901.keys())
print(vars)
tcr_JJA_corrs_masked=[]

for var in vars:

    x=detrend_dim(enlarged_tcr, dim="time", deg=1).sel(time=slice(startyear,endyear))
    print(x.shape)
    y=detrend_dim(tcrv3_JJA_anoms1901[var], dim="time", deg=1).sel(time=slice(startyear,endyear))
    print(y.shape)
    xr_R,xr_R_masked,xr_p=corr_with_sig(x,y,sig_level=0.05,lon_name="lon",lat_name="lat")
    # for some fucking reason we have to reassign the coordinates in the 20CRv3 case
    xr_R_masked["lon"]=tcrv3_JJA_anoms1901[var]["lon"]
    xr_R_masked["lat"]=tcrv3_JJA_anoms1901[var]["lat"]
    tcr_JJA_corrs_masked.append(xr_R_masked)
    
    fig = plt.figure(figsize=(8, 4), dpi= 200)
    ax = plt.axes(projection=ccrs.EqualEarth(central_longitude=0, globe=None)) 
    tplot=xr_R_masked.plot.pcolormesh(ax=ax,
    levels = 17, transform=ccrs.PlateCarree(), vmax=0.6, cmap=current_cmap, cbar_kwargs={'orientation':'vertical',
    'fraction':0.012, 'pad':0.015, 'aspect':35})

    tplot.colorbar.set_label('Pearson Corr. Coef.', size=10) 
    tplot.ylabel_style = {'size':16}
    
    ax.set_global()
    ax.coastlines()
    ax.set_title(var+"_"+startyear+endyear)
    ax.set_extent([-90, 40, 25, 80])
    ;
    fig.savefig(plot_folder+"o18pc1_JJA_CORR_20crv3_"+var+"_"+startyear+endyear+".pdf")

In [ ]:
choice=[0,1,2,4]
ds_merged=xr.concat(tcr_JJA_corrs_masked,dim="vars")
ds_merged=ds_merged.isel(vars=choice)


seasons=list(["t2m","slp","precip","v250"])
INDEX=list(["(a)","(b)","(c)","(d)"])
fig, axs = plt.subplots(2, 2, figsize=(8, 9), dpi= 300,subplot_kw={'projection': ccrs.EqualEarth(central_longitude=0, globe=None)})
# axs is a 2 dimensional array of `GeoAxes`.  We will flatten it into a 1-D array
axs=axs.flatten()

#Loop over all of the models
for i in range(4):
    tplot=ds_merged[i,:,:].plot.pcolormesh(ax=axs[i],levels = 17,vmax=0.6, transform=ccrs.PlateCarree(), cmap=current_cmap,add_colorbar=False)
    axs[i].set_title(INDEX[i],loc='left')
    axs[i].coastlines()
    axs[i].set_title(seasons[i])
    axs[i].set_extent([-90, 50, 25, 80])
    
# Adjust the location of the subplots on the page to make room for the colorbar
fig.subplots_adjust(bottom=0.25, top=0.9, left=0.2, right=0.8,
                    wspace=0.1, hspace=0.01)

# Add a colorbar axis at the bottom of the graph
cbar_ax = fig.add_axes([0.2, 0.2, 0.6, 0.02])

# Draw the colorbar
cbar=fig.colorbar(tplot, cax=cbar_ax,orientation='horizontal')
cbar.set_label('Pearson Corr Coeff', size=10) 




fig.savefig(plot_folder+"o18pc1_JJA_CORR_20crv3_"+startyear+endyear+".pdf")

## 20cr comparison wth EKF400

In [ ]:
startyear="1836"
endyear="2003"
vars=list(ekf_JJA_anoms1901.keys())
print(vars)
ekf_JJA_corrs_masked=[]

for var in vars:

    x=detrend_dim(enlarged_ekf, dim="time", deg=1).sel(time=slice(startyear,endyear))
    y=detrend_dim(ekf_JJA_anoms1901[var], dim="time", deg=1).sel(time=slice(startyear,endyear))
    xr_R,xr_R_masked,xr_p=corr_with_sig(x,y,sig_level=0.05)
    ekf_JJA_corrs_masked.append(xr_R_masked)


In [ ]:
vars=list(tcrv3_JJA_anoms1901.keys())
print(vars)
tcr_JJA_corrs_masked=[]

for var in vars:

    x=detrend_dim(enlarged_tcr, dim="time", deg=1).sel(time=slice(startyear,endyear))

    y=detrend_dim(tcrv3_JJA_anoms1901[var], dim="time", deg=1).sel(time=slice(startyear,endyear))

    xr_R,xr_R_masked,xr_p=corr_with_sig(x,y,sig_level=0.05,lon_name="lon",lat_name="lat")
    # for some fucking reason we have to reassign the coordinates in the 20CRv3 case
    xr_R_masked["lon"]=tcrv3_JJA_anoms1901[var]["lon"]
    xr_R_masked["lat"]=tcrv3_JJA_anoms1901[var]["lat"]
    tcr_JJA_corrs_masked.append(xr_R_masked)

In [ ]:
ds_merged1=xr.concat(tcr_JJA_corrs_masked,dim="vars")
choice=[0,1,2,4]
ds_merged11=ds_merged1.isel(vars=choice)
ds_merged2=xr.concat(ekf_JJA_corrs_masked,dim="vars")
choice=[0,2,1,4]

In [ ]:
ds_merged22= ds_merged2.rename({'longitude': 'lon','latitude': 'lat'})
ds_merged11_regrid=ds_merged11.regrid.linear(ds_merged22)
ds_merged11_regrid=ds_merged11_regrid.drop(["longitude","latitude"])

In [ ]:
comparison=xr.concat([ds_merged11_regrid[0,:,:],ds_merged22[0,:,:],ds_merged11_regrid[1,:,:],ds_merged22[2,:,:],ds_merged11_regrid[2,:,:],ds_merged22[1,:,:],ds_merged11_regrid[3,:,:],ds_merged22[4,:,:]],dim="vars")

In [ ]:

seasons=list(["t2m","t2m","slp","slp","precip","precip","v250","v250"])
INDEX=list(["(a)","(b)","(c)","(d)","(e)","(f)","(g)","(h)"])
fig, axs = plt.subplots(4, 2, figsize=(10, 9), dpi= 300,subplot_kw={'projection': ccrs.EqualEarth(central_longitude=0, globe=None)})
# axs is a 2 dimensional array of `GeoAxes`.  We will flatten it into a 1-D array
axs=axs.flatten()

#Loop over all of the models
for i in range(8):
    tplot=comparison[i,:,:].plot.pcolormesh(ax=axs[i],levels = 17,vmax=0.6, transform=ccrs.PlateCarree(), cmap=current_cmap,add_colorbar=False)
    axs[i].set_title(INDEX[i],loc='left')
    axs[i].coastlines()
    axs[i].set_title(seasons[i])
    axs[i].set_extent([-90, 50, 25, 80])

# Adjust the location of the subplots on the page to make room for the colorbar
fig.subplots_adjust(bottom=0.25, top=0.9, left=0.2, right=0.8,
                    wspace=0.1, hspace=0.01)

# Add a colorbar axis at the bottom of the graph
cbar_ax = fig.add_axes([0.2, 0.2, 0.6, 0.02])

# Draw the colorbar
cbar=fig.colorbar(tplot, cax=cbar_ax,orientation='horizontal')
cbar.set_label('Pearson Corr Coeff', size=10) 




fig.savefig(plot_folder+"o18pc1_JJA_CORR_compare_20crv3_EKF_"+startyear+endyear+".pdf")

## CERA20C

In [ ]:
startyear="1901"
endyear="2010"
vars=list(cera20c_JJA_anoms1901.keys())
print(vars)
cera20c_JJA_corrs_masked=[]

for var in vars:

    x=detrend_dim(enlarged_cera20c, dim="time", deg=1).sel(time=slice(startyear,endyear))
    y=detrend_dim(cera20c_JJA_anoms1901[var], dim="time", deg=1).sel(time=slice(startyear,endyear))
    xr_R,xr_R_masked,xr_p=corr_with_sig(x,y,sig_level=0.05)
    cera20c_JJA_corrs_masked.append(xr_R_masked)
    
    fig = plt.figure(figsize=(8, 4), dpi= 200)
    ax = plt.axes(projection=ccrs.EqualEarth(central_longitude=0, globe=None)) 
    tplot=xr_R_masked.plot.pcolormesh(ax=ax,
    levels = 17, transform=ccrs.PlateCarree(), vmax=0.6, cmap=current_cmap, cbar_kwargs={'orientation':'vertical',
    'fraction':0.012, 'pad':0.015, 'aspect':35})

    tplot.colorbar.set_label('Pearson Corr. Coef.', size=10) 
    tplot.ylabel_style = {'size':16}
    
    ax.set_global()
    ax.coastlines()
    ax.set_title(var+"_"+startyear+endyear)
    ax.set_extent([-90, 40, 25, 80])
    ;
    fig.savefig(plot_folder+"o18pc1_JJA_CORR_cera20c_"+var+"_"+startyear+endyear+".pdf")

In [ ]:
choice=[3,2,4,1]
ds_merged=xr.concat(cera20c_JJA_corrs_masked,dim="vars")
ds_merged=ds_merged.isel(vars=choice)


seasons=list(["t2m","slp","precip","v250"])
INDEX=list(["(a)","(b)","(c)","(d)"])
fig, axs = plt.subplots(2, 2, figsize=(8, 9), dpi= 300,subplot_kw={'projection': ccrs.EqualEarth(central_longitude=0, globe=None)})
# axs is a 2 dimensional array of `GeoAxes`.  We will flatten it into a 1-D array
axs=axs.flatten()

#Loop over all of the models
for i in range(4):
    tplot=ds_merged[i,:,:].plot.pcolormesh(ax=axs[i],levels = 17,vmax=0.6, transform=ccrs.PlateCarree(), cmap=current_cmap,add_colorbar=False)
    axs[i].set_title(INDEX[i],loc='left')
    axs[i].coastlines()
    axs[i].set_title(seasons[i])
    axs[i].set_extent([-90, 50, 25, 80])
    
# Adjust the location of the subplots on the page to make room for the colorbar
fig.subplots_adjust(bottom=0.25, top=0.9, left=0.2, right=0.8,
                    wspace=0.1, hspace=0.01)

# Add a colorbar axis at the bottom of the graph
cbar_ax = fig.add_axes([0.2, 0.2, 0.6, 0.02])

# Draw the colorbar
cbar=fig.colorbar(tplot, cax=cbar_ax,orientation='horizontal')
cbar.set_label('Pearson Corr Coeff', size=10) 




fig.savefig(plot_folder+"o18pc1_JJA_CORR_cera20c_"+startyear+endyear+".pdf")

## cera20c comparison wth EKF400

In [ ]:
startyear="1901"
endyear="2003"
vars=list(ekf_JJA_anoms1901.keys())
print(vars)
ekf_JJA_corrs_masked=[]

for var in vars:

    x=detrend_dim(enlarged_ekf, dim="time", deg=1).sel(time=slice(startyear,endyear))
    y=detrend_dim(ekf_JJA_anoms1901[var], dim="time", deg=1).sel(time=slice(startyear,endyear))
    xr_R,xr_R_masked,xr_p=corr_with_sig(x,y,sig_level=0.05)
    ekf_JJA_corrs_masked.append(xr_R_masked)


In [ ]:
startyear="1901"
endyear="2003"
vars=list(cera20c_JJA_anoms1901.keys())
print(vars)
cera20c_JJA_corrs_masked=[]

for var in vars:

    x=detrend_dim(enlarged_cera20c, dim="time", deg=1).sel(time=slice(startyear,endyear))
    y=detrend_dim(cera20c_JJA_anoms1901[var], dim="time", deg=1).sel(time=slice(startyear,endyear))
    xr_R,xr_R_masked,xr_p=corr_with_sig(x,y,sig_level=0.05)
    cera20c_JJA_corrs_masked.append(xr_R_masked)
    

In [ ]:
ds_merged1=xr.concat(cera20c_JJA_corrs_masked,dim="vars")
choice=[3,2,4,1]
ds_merged11=ds_merged1.isel(vars=choice)
ds_merged2=xr.concat(ekf_JJA_corrs_masked,dim="vars")
choice=[0,2,1,4]
ds_merged22=ds_merged2

In [ ]:
ds_merged11_regrid=ds_merged11.regrid.linear(ds_merged2)
#ds_merged11_regrid=ds_merged11_regrid.drop(["longitude","latitude"])

In [ ]:
comparison=xr.concat([ds_merged11_regrid[0,:,:],ds_merged22[0,:,:],ds_merged11_regrid[1,:,:],ds_merged22[2,:,:],ds_merged11_regrid[2,:,:],ds_merged22[1,:,:],ds_merged11_regrid[3,:,:],ds_merged22[4,:,:]],dim="vars")

In [ ]:

seasons=list(["t2m","t2m","slp","slp","precip","precip","v250","v250"])
INDEX=list(["(a)","(b)","(c)","(d)","(e)","(f)","(g)","(h)"])
fig, axs = plt.subplots(4, 2, figsize=(10, 9), dpi= 300,subplot_kw={'projection': ccrs.EqualEarth(central_longitude=0, globe=None)})
# axs is a 2 dimensional array of `GeoAxes`.  We will flatten it into a 1-D array
axs=axs.flatten()

#Loop over all of the models
for i in range(8):
    tplot=comparison[i,:,:].plot.pcolormesh(ax=axs[i],levels = 17,vmax=0.6, transform=ccrs.PlateCarree(), cmap=current_cmap,add_colorbar=False)
    axs[i].set_title(INDEX[i],loc='left')
    axs[i].coastlines()
    axs[i].set_title(seasons[i])
    axs[i].set_extent([-90, 50, 25, 80])

# Adjust the location of the subplots on the page to make room for the colorbar
fig.subplots_adjust(bottom=0.25, top=0.9, left=0.2, right=0.8,
                    wspace=0.1, hspace=0.01)

# Add a colorbar axis at the bottom of the graph
cbar_ax = fig.add_axes([0.2, 0.2, 0.6, 0.02])

# Draw the colorbar
cbar=fig.colorbar(tplot, cax=cbar_ax,orientation='horizontal')
cbar.set_label('Pearson Corr Coeff', size=10) 




fig.savefig(plot_folder+"o18pc1_JJA_CORR_compare_cera20c_EKF_"+startyear+endyear+".pdf")

## modera

In [ ]:
startyear="1600"
endyear="2008"
vars=list(modera_JJA_anoms1901.keys())
print(vars)
modera_JJA_corrs_masked=[]

for var in vars:

    x=detrend_dim(enlarged_modera, dim="time", deg=1).sel(time=slice(startyear,endyear))
    y=detrend_dim(modera_JJA_anoms1901[var], dim="time", deg=1).sel(time=slice(startyear,endyear))
    xr_R,xr_R_masked,xr_p=corr_with_sig(x,y,sig_level=0.05)
    modera_JJA_corrs_masked.append(xr_R_masked)
    
    fig = plt.figure(figsize=(8, 4), dpi= 200)
    ax = plt.axes(projection=ccrs.EqualEarth(central_longitude=0, globe=None)) 
    tplot=xr_R_masked.plot.pcolormesh(ax=ax,
    levels = 17, transform=ccrs.PlateCarree(), vmax=0.6, cmap=current_cmap, cbar_kwargs={'orientation':'vertical',
    'fraction':0.012, 'pad':0.015, 'aspect':35})

    tplot.colorbar.set_label('Pearson Corr. Coef.', size=10) 
    tplot.ylabel_style = {'size':16}
    
    ax.set_global()
    ax.coastlines()
    ax.set_title(var+"_"+startyear+endyear)
    ax.set_extent([-90, 40, 25, 80])
    ;
    fig.savefig(plot_folder+"o18pc1_JJA_CORR_modera_"+var+"_"+startyear+endyear+".pdf")

In [ ]:
startyear="1600"
endyear="2008"
vars=list(modera_jan_anoms1901.keys())
print(vars)
modera_jan_corrs_masked=[]

for var in vars:

    x=detrend_dim(enlarged_modera, dim="time", deg=1).sel(time=slice(startyear,endyear))
    y=detrend_dim(modera_jan_anoms1901[var], dim="time", deg=1).sel(time=slice(startyear,endyear))
    xr_R,xr_R_masked,xr_p=corr_with_sig(x,y,sig_level=0.05)
    modera_jan_corrs_masked.append(xr_R_masked)
    
    fig = plt.figure(figsize=(8, 4), dpi= 200)
    ax = plt.axes(projection=ccrs.EqualEarth(central_longitude=0, globe=None)) 
    tplot=xr_R_masked.plot.pcolormesh(ax=ax,
    levels = 17, transform=ccrs.PlateCarree(), vmax=0.6, cmap=current_cmap, cbar_kwargs={'orientation':'vertical',
    'fraction':0.012, 'pad':0.015, 'aspect':35})

    tplot.colorbar.set_label('Pearson Corr. Coef.', size=10) 
    tplot.ylabel_style = {'size':16}
    
    ax.set_global()
    ax.coastlines()
    ax.set_title(var+"_"+startyear+endyear)
    ax.set_extent([-90, 40, 25, 80])
    ;
    fig.savefig(plot_folder+"o18pc1_jan_CORR_modera_"+var+"_"+startyear+endyear+".pdf")

In [ ]:
startyear="1600"
endyear="2008"
vars=list(modera_feb_anoms1901.keys())
print(vars)
modera_feb_corrs_masked=[]

for var in vars:

    x=detrend_dim(enlarged_modera, dim="time", deg=1).sel(time=slice(startyear,endyear))
    y=detrend_dim(modera_feb_anoms1901[var], dim="time", deg=1).sel(time=slice(startyear,endyear))
    xr_R,xr_R_masked,xr_p=corr_with_sig(x,y,sig_level=0.05)
    modera_feb_corrs_masked.append(xr_R_masked)
    
    fig = plt.figure(figsize=(8, 4), dpi= 200)
    ax = plt.axes(projection=ccrs.EqualEarth(central_longitude=0, globe=None)) 
    tplot=xr_R_masked.plot.pcolormesh(ax=ax,
    levels = 17, transform=ccrs.PlateCarree(), vmax=0.6, cmap=current_cmap, cbar_kwargs={'orientation':'vertical',
    'fraction':0.012, 'pad':0.015, 'aspect':35})

    tplot.colorbar.set_label('Pearson Corr. Coef.', size=10) 
    tplot.ylabel_style = {'size':16}
    
    ax.set_global()
    ax.coastlines()
    ax.set_title(var+"_"+startyear+endyear)
    ax.set_extent([-90, 40, 25, 80])
    ;
    fig.savefig(plot_folder+"o18pc1_feb_CORR_modera_"+var+"_"+startyear+endyear+".pdf")

## modera comparison with ekf400

In [ ]:
startyear="1901"
endyear="2003"
vars=list(ekf_JJA_anoms1901.keys())
print(vars)
ekf_JJA_corrs_masked=[]

for var in vars:

    x=detrend_dim(enlarged_ekf, dim="time", deg=1).sel(time=slice(startyear,endyear))
    y=detrend_dim(ekf_JJA_anoms1901[var], dim="time", deg=1).sel(time=slice(startyear,endyear))
    xr_R,xr_R_masked,xr_p=corr_with_sig(x,y,sig_level=0.05)
    ekf_JJA_corrs_masked.append(xr_R_masked)


In [ ]:
startyear="1901"
endyear="2003"
vars=list(modera_JJA_anoms1901.keys())
print(vars)
modera_JJA_corrs_masked=[]

for var in vars:

    x=detrend_dim(enlarged_modera, dim="time", deg=1).sel(time=slice(startyear,endyear))
    y=detrend_dim(modera_JJA_anoms1901[var], dim="time", deg=1).sel(time=slice(startyear,endyear))
    xr_R,xr_R_masked,xr_p=corr_with_sig(x,y,sig_level=0.05)
    modera_JJA_corrs_masked.append(xr_R_masked)

In [ ]:
ds_merged1=xr.concat(modera_JJA_corrs_masked,dim="vars")
choice=[0,1,2]
ds_merged11=ds_merged1.isel(vars=choice)
ds_merged2=xr.concat(ekf_JJA_corrs_masked,dim="vars")
choice=[0,2,1]
ds_merged22=ds_merged2

In [ ]:
ds_merged11_regrid=ds_merged11.regrid.linear(ds_merged2)
#ds_merged11_regrid=ds_merged11_regrid.drop(["longitude","latitude"])
#ds_merged11_regrid=ds_merged11

In [ ]:
comparison=xr.concat([ds_merged11_regrid[0,:,:],ds_merged22[0,:,:],ds_merged11_regrid[1,:,:],ds_merged22[2,:,:],ds_merged11_regrid[2,:,:],ds_merged22[1,:,:],ds_merged11_regrid[0,:,:],ds_merged22[0,:,:]],dim="vars")

In [ ]:

seasons=list(["t2m","t2m","slp","slp","precip","precip"])
INDEX=list(["(a)","(b)","(c)","(d)","(e)","(f)"])
fig, axs = plt.subplots(3, 2, figsize=(10, 9), dpi= 300,subplot_kw={'projection': ccrs.EqualEarth(central_longitude=0, globe=None)})
# axs is a 2 dimensional array of `GeoAxes`.  We will flatten it into a 1-D array
axs=axs.flatten()

#Loop over all of the models
for i in range(6):
    tplot=comparison[i,:,:].plot.pcolormesh(ax=axs[i],levels = 17,vmax=0.6, transform=ccrs.PlateCarree(), cmap=current_cmap,add_colorbar=False)
    axs[i].set_title(INDEX[i],loc='left')
    axs[i].coastlines()
    axs[i].set_title(seasons[i])
    axs[i].set_extent([-90, 50, 25, 80])

# Adjust the location of the subplots on the page to make room for the colorbar
fig.subplots_adjust(bottom=0.25, top=0.9, left=0.2, right=0.8,
                    wspace=0.1, hspace=0.01)

# Add a colorbar axis at the bottom of the graph
cbar_ax = fig.add_axes([0.2, 0.2, 0.6, 0.02])

# Draw the colorbar
cbar=fig.colorbar(tplot, cax=cbar_ax,orientation='horizontal')
cbar.set_label('Pearson Corr Coeff', size=10) 




fig.savefig(plot_folder+"o18pc1_JJA_CORR_compare_modera_EKF_"+startyear+endyear+".pdf")

In [ ]:
# jan

In [ ]:
startyear="1602"
endyear="2003"
vars=list(ekf_jan_anoms1901.keys())
print(vars)
ekf_jan_corrs_masked=[]

for var in vars:

    x=detrend_dim(enlarged_ekf, dim="time", deg=1).sel(time=slice(startyear,endyear))
    y=detrend_dim(ekf_jan_anoms1901[var], dim="time", deg=1).sel(time=slice(startyear,endyear))
    xr_R,xr_R_masked,xr_p=corr_with_sig(x,y,sig_level=0.05)
    ekf_jan_corrs_masked.append(xr_R_masked)


In [ ]:
startyear="1602"
endyear="2003"
vars=list(modera_jan_anoms1901.keys())
print(vars)
modera_jan_corrs_masked=[]

for var in vars:

    x=detrend_dim(enlarged_modera, dim="time", deg=1).sel(time=slice(startyear,endyear))
    y=detrend_dim(modera_jan_anoms1901[var], dim="time", deg=1).sel(time=slice(startyear,endyear))
    xr_R,xr_R_masked,xr_p=corr_with_sig(x,y,sig_level=0.05)
    modera_jan_corrs_masked.append(xr_R_masked)

In [ ]:
ds_merged1=xr.concat(modera_jan_corrs_masked,dim="vars")
choice=[0,1,2]
ds_merged11=ds_merged1.isel(vars=choice)
ds_merged2=xr.concat(ekf_jan_corrs_masked,dim="vars")
choice=[0,2,1]
ds_merged22=ds_merged2

In [ ]:
ds_merged11_regrid=ds_merged11.regrid.linear(ds_merged2)
#ds_merged11_regrid=ds_merged11_regrid.drop(["longitude","latitude"])
#ds_merged11_regrid=ds_merged11

In [ ]:
comparison=xr.concat([ds_merged11_regrid[0,:,:],ds_merged22[0,:,:],ds_merged11_regrid[1,:,:],ds_merged22[2,:,:],ds_merged11_regrid[2,:,:],ds_merged22[1,:,:],ds_merged11_regrid[0,:,:],ds_merged22[0,:,:]],dim="vars")

In [ ]:

seasons=list(["t2m","t2m","slp","slp","precip","precip"])
INDEX=list(["(a)","(b)","(c)","(d)","(e)","(f)"])
fig, axs = plt.subplots(3, 2, figsize=(10, 9), dpi= 300,subplot_kw={'projection': ccrs.EqualEarth(central_longitude=0, globe=None)})
# axs is a 2 dimensional array of `GeoAxes`.  We will flatten it into a 1-D array
axs=axs.flatten()

#Loop over all of the models
for i in range(6):
    tplot=comparison[i,:,:].plot.pcolormesh(ax=axs[i],levels = 17,vmax=0.6, transform=ccrs.PlateCarree(), cmap=current_cmap,add_colorbar=False)
    axs[i].set_title(INDEX[i],loc='left')
    axs[i].coastlines()
    axs[i].set_title(seasons[i])
    axs[i].set_extent([-90, 50, 25, 80])

# Adjust the location of the subplots on the page to make room for the colorbar
fig.subplots_adjust(bottom=0.25, top=0.9, left=0.2, right=0.8,
                    wspace=0.1, hspace=0.01)

# Add a colorbar axis at the bottom of the graph
cbar_ax = fig.add_axes([0.2, 0.2, 0.6, 0.02])

# Draw the colorbar
cbar=fig.colorbar(tplot, cax=cbar_ax,orientation='horizontal')
cbar.set_label('Pearson Corr Coeff', size=10) 




fig.savefig(plot_folder+"o18pc1_jan_CORR_compare_modera_EKF_"+startyear+endyear+".pdf")

In [ ]:
# feb

In [ ]:
startyear="1602"
endyear="2003"
vars=list(ekf_feb_anoms1901.keys())
print(vars)
ekf_feb_corrs_masked=[]

for var in vars:

    x=detrend_dim(enlarged_ekf, dim="time", deg=1).sel(time=slice(startyear,endyear))
    y=detrend_dim(ekf_feb_anoms1901[var], dim="time", deg=1).sel(time=slice(startyear,endyear))
    xr_R,xr_R_masked,xr_p=corr_with_sig(x,y,sig_level=0.05)
    ekf_feb_corrs_masked.append(xr_R_masked)


In [ ]:
startyear="1602"
endyear="2003"
vars=list(modera_feb_anoms1901.keys())
print(vars)
modera_feb_corrs_masked=[]

for var in vars:

    x=detrend_dim(enlarged_modera, dim="time", deg=1).sel(time=slice(startyear,endyear))
    y=detrend_dim(modera_feb_anoms1901[var], dim="time", deg=1).sel(time=slice(startyear,endyear))
    xr_R,xr_R_masked,xr_p=corr_with_sig(x,y,sig_level=0.05)
    modera_feb_corrs_masked.append(xr_R_masked)

In [ ]:
ds_merged1=xr.concat(modera_feb_corrs_masked,dim="vars")
choice=[0,1,2]
ds_merged11=ds_merged1.isel(vars=choice)
ds_merged2=xr.concat(ekf_feb_corrs_masked,dim="vars")
choice=[0,2,1]
ds_merged22=ds_merged2

In [ ]:
ds_merged11_regrid=ds_merged11.regrid.linear(ds_merged2)
#ds_merged11_regrid=ds_merged11_regrid.drop(["longitude","latitude"])
#ds_merged11_regrid=ds_merged11

In [ ]:
comparison=xr.concat([ds_merged11_regrid[0,:,:],ds_merged22[0,:,:],ds_merged11_regrid[1,:,:],ds_merged22[2,:,:],ds_merged11_regrid[2,:,:],ds_merged22[1,:,:],ds_merged11_regrid[0,:,:],ds_merged22[0,:,:]],dim="vars")

In [ ]:

seasons=list(["t2m","t2m","slp","slp","precip","precip"])
INDEX=list(["(a)","(b)","(c)","(d)","(e)","(f)"])
fig, axs = plt.subplots(3, 2, figsize=(10, 9), dpi= 300,subplot_kw={'projection': ccrs.EqualEarth(central_longitude=0, globe=None)})
# axs is a 2 dimensional array of `GeoAxes`.  We will flatten it into a 1-D array
axs=axs.flatten()

#Loop over all of the models
for i in range(6):
    tplot=comparison[i,:,:].plot.pcolormesh(ax=axs[i],levels = 17,vmax=0.6, transform=ccrs.PlateCarree(), cmap=current_cmap,add_colorbar=False)
    axs[i].set_title(INDEX[i],loc='left')
    axs[i].coastlines()
    axs[i].set_title(seasons[i])
    axs[i].set_extent([-90, 50, 25, 80])

# Adjust the location of the subplots on the page to make room for the colorbar
fig.subplots_adjust(bottom=0.25, top=0.9, left=0.2, right=0.8,
                    wspace=0.1, hspace=0.01)

# Add a colorbar axis at the bottom of the graph
cbar_ax = fig.add_axes([0.2, 0.2, 0.6, 0.02])

# Draw the colorbar
cbar=fig.colorbar(tplot, cax=cbar_ax,orientation='horizontal')
cbar.set_label('Pearson Corr Coeff', size=10) 




fig.savefig(plot_folder+"o18pc1_feb_CORR_compare_modera_EKF_"+startyear+endyear+".pdf")

# lag Correlations with SSTS

In [ ]:
ssts_member=xr.open_mfdataset(sst_folder+"PaleoSST_1600-1849_R*_AM_remap.nc", combine='nested',concat_dim="member")

In [ ]:
ssts_member_AM_anoms1901=create_anomalies_alt(da=ssts_member,month_or_season=5,ref_time=["1610-01-01","1800-12-31"])


In [ ]:
len(ssts_member_AM_anoms1901.member)

In [ ]:
ssts_member_AM_anoms1901=ssts_member_AM_anoms1901.resample(time="1Y").sum()

In [ ]:
ssts=xr.open_dataset(sst_folder+"PaleoSST_1600-2010_mergensmean_AM_remap.nc")

In [ ]:
ssts_AM_anoms1901=create_anomalies_alt(da=ssts,month_or_season=5,ref_time=["1901-01-01","2000-12-31"])


In [ ]:
enlarged_ssts=enlarge_grid(pc1df_ssts.values,ssts.sst,"PC1 o18 new")

In [ ]:
startyear="1781"
endyear="1890"
vars=list(ssts_AM_anoms1901.keys())
print(vars)
ssts_JJA_corrs_masked=[]

for var in vars:

    x=detrend_dim(enlarged_ssts, dim="time", deg=1).sel(time=slice(startyear,endyear))
    y=detrend_dim(ssts_AM_anoms1901[var].fillna(0), dim="time", deg=1).sel(time=slice(startyear,endyear))
    xr_R,xr_R_masked,xr_p=corr_with_sig(x,y,sig_level=0.05)
    ssts_JJA_corrs_masked.append(xr_R_masked)
    
    fig = plt.figure(figsize=(8, 4), dpi= 200)
    ax = plt.axes(projection=ccrs.EqualEarth(central_longitude=0, globe=None)) 
    tplot=xr_R_masked.plot.pcolormesh(ax=ax,
    levels = 17, transform=ccrs.PlateCarree(), vmax=0.6, cmap=current_cmap, cbar_kwargs={'orientation':'vertical',
    'fraction':0.012, 'pad':0.015, 'aspect':35})

    tplot.colorbar.set_label('Correlation', size=10) 
    tplot.ylabel_style = {'size':16}
    
    ax.set_global()
    ax.coastlines()
    ax.set_title(var+"_"+startyear+endyear)
    #ax.set_extent([-90, 40, 25, 80])
    ;
    fig.savefig(plot_folder+"o18pc1_JJA_CORR_mergedssts_"+var+"_"+startyear+endyear+"_2025.pdf")

In [ ]:
startyear="1851"
endyear="2010"
vars=list(ssts_AM_anoms1901.keys())
print(vars)
ssts_JJA_corrs_masked=[]

for var in vars:

    x=detrend_dim(enlarged_ssts, dim="time", deg=1).sel(time=slice(startyear,endyear))
    y=detrend_dim(ssts_AM_anoms1901[var].fillna(0), dim="time", deg=1).sel(time=slice(startyear,endyear))
    xr_R,xr_R_masked,xr_p=corr_with_sig(x,y,sig_level=0.99)
    ssts_JJA_corrs_masked.append(xr_R_masked)
    
    fig = plt.figure(figsize=(8, 4), dpi= 200)
    ax = plt.axes(projection=ccrs.EqualEarth(central_longitude=0, globe=None)) 
    tplot=xr_R.plot.pcolormesh(ax=ax,
    levels = 17, transform=ccrs.PlateCarree(), vmax=0.6, cmap=current_cmap, cbar_kwargs={'orientation':'vertical',
    'fraction':0.012, 'pad':0.015, 'aspect':35})

    tplot.colorbar.set_label('Pearson Corr. Coef.', size=10) 
    tplot.ylabel_style = {'size':16}
    
    ax.set_global()
    ax.coastlines()
    ax.set_title(var+"_"+startyear+endyear)
    #ax.set_extent([-90, 40, 25, 80])
    ;
    fig.savefig(plot_folder+"o18pc1_JJA_CORR_mergedssts_"+var+"_"+startyear+endyear+"_fullfield.pdf")

fullfield=xr_R

In [ ]:
x=detrend_dim(enlarged_ssts, dim="time", deg=1).sel(time=slice("1851","2010"))
y=detrend_dim(ssts_AM_anoms1901[var].fillna(0), dim="time", deg=1).sel(time=slice("1851","2010"))
xr_R,ds7_var,xr_p=corr_with_sig(x,y,sig_level=0.05)


x=detrend_dim(enlarged_ssts, dim="time", deg=1).sel(time=slice("1901","2010"))
y=detrend_dim(ssts_AM_anoms1901[var].fillna(0), dim="time", deg=1).sel(time=slice("1901","2010"))
xr_R,ds4_var,xr_p=corr_with_sig(x,y,sig_level=0.05)

x=detrend_dim(enlarged_ssts, dim="time", deg=1).sel(time=slice("1851","1960"))
y=detrend_dim(ssts_AM_anoms1901[var].fillna(0), dim="time", deg=1).sel(time=slice("1851","1960"))
xr_R,ds5_var,xr_p=corr_with_sig(x,y,sig_level=0.05)

x=detrend_dim(enlarged_ssts, dim="time", deg=1).sel(time=slice("1781","1890"))
y=detrend_dim(ssts_AM_anoms1901[var].fillna(0), dim="time", deg=1).sel(time=slice("1781","1890"))
xr_R,ds6_var,xr_p=corr_with_sig(x,y,sig_level=0.05)




ds_merged=xr.concat([ds7_var,ds4_var,ds5_var,ds6_var],dim="height")





seasons=list(["1851-2010","1901-2010","1851-1960","1781-1890"])
INDEX=list(["(a)","(b)","(c)","(d)"])
fig, axs = plt.subplots(2, 2, figsize=(10, 5), dpi= 300,subplot_kw={'projection': ccrs.EqualEarth(central_longitude=0, globe=None)})
# axs is a 2 dimensional array of `GeoAxes`.  We will flatten it into a 1-D array
axs=axs.flatten()

#Loop over all of the models
for i in range(4):
    tplot=ds_merged[i,:,:].plot.pcolormesh(ax=axs[i],levels = 17, vmax=0.6, transform=ccrs.PlateCarree(), cmap=current_cmap,add_colorbar=False)
    axs[i].set_title(INDEX[i],loc='left')
    axs[i].coastlines()
    axs[i].set_title(seasons[i])

# Adjust the location of the subplots on the page to make room for the colorbar
fig.subplots_adjust(bottom=0.25, top=0.9, left=0.2, right=0.8,
                    wspace=0.1, hspace=0.01)

# Add a colorbar axis at the bottom of the graph
cbar_ax = fig.add_axes([0.2, 0.2, 0.6, 0.02])

# Draw the colorbar
cbar=fig.colorbar(tplot, cax=cbar_ax,orientation='horizontal')
cbar.set_label('Pearson Corr Coeff', size=10) 




# Make it nice
#plt.tight_layout()
fig.savefig(plot_folder+"sst_correlation_recently.pdf")

In [ ]:
x=detrend_dim(enlarged_ssts, dim="time", deg=1).sel(time=slice("1601","2010"))
y=detrend_dim(ssts_AM_anoms1901[var].fillna(0), dim="time", deg=1).sel(time=slice("1601","2010"))
xr_R,ds7_var,xr_p=corr_with_sig(x,y,sig_level=0.05)

x=detrend_dim(enlarged_ssts, dim="time", deg=1).sel(time=slice("1601","1700"))
y=detrend_dim(ssts_AM_anoms1901[var].fillna(0), dim="time", deg=1).sel(time=slice("1601","1700"))
xr_R,ds8_var,xr_p=corr_with_sig(x,y,sig_level=0.05)

x=detrend_dim(enlarged_ssts, dim="time", deg=1).sel(time=slice("1701","1800"))
y=detrend_dim(ssts_AM_anoms1901[var].fillna(0), dim="time", deg=1).sel(time=slice("1701","1800"))
xr_R,ds4_var,xr_p=corr_with_sig(x,y,sig_level=0.05)

x=detrend_dim(enlarged_ssts, dim="time", deg=1).sel(time=slice("1801","1900"))
y=detrend_dim(ssts_AM_anoms1901[var].fillna(0), dim="time", deg=1).sel(time=slice("1801","1900"))
xr_R,ds5_var,xr_p=corr_with_sig(x,y,sig_level=0.05)

x=detrend_dim(enlarged_ssts, dim="time", deg=1).sel(time=slice("1901","2000"))
y=detrend_dim(ssts_AM_anoms1901[var].fillna(0), dim="time", deg=1).sel(time=slice("1901","2000"))
xr_R,ds6_var,xr_p=corr_with_sig(x,y,sig_level=0.05)




ds_merged=xr.concat([ds7_var,ds8_var,ds4_var,ds5_var,ds6_var],dim="height")





seasons=list(["1601-2010","1601-1700","1701-1800","1801-1900","1901-2000"])
INDEX=list(["(a)","(b)","(c)","(d)","(e)"])
fig, axs = plt.subplots(5, 1, figsize=(6.5, 17), dpi= 300,subplot_kw={'projection': ccrs.EqualEarth(central_longitude=0, globe=None)})
# axs is a 2 dimensional array of `GeoAxes`.  We will flatten it into a 1-D array
axs=axs.flatten()

#Loop over all of the models
for i in range(5):
    tplot=ds_merged[i,:,:].plot.pcolormesh(ax=axs[i],levels = 17, vmax=0.6, transform=ccrs.PlateCarree(), cmap=current_cmap,add_colorbar=False)
    axs[i].set_title(INDEX[i],loc='left')
    axs[i].coastlines()
    axs[i].set_title(seasons[i])

# Adjust the location of the subplots on the page to make room for the colorbar
fig.subplots_adjust(bottom=0.25, top=0.9, left=0.2, right=0.8,
                    wspace=0.1, hspace=0.1)

# Add a colorbar axis at the bottom of the graph
cbar_ax = fig.add_axes([0.2, 0.2, 0.6, 0.02])

# Draw the colorbar
cbar=fig.colorbar(tplot, cax=cbar_ax,orientation='horizontal')
cbar.set_label('Pearson Corr Coeff', size=10) 




# Make it nice
#plt.tight_layout()
fig.savefig(plot_folder+"sst_correlation_all_centuries_005.pdf")

In [ ]:
x=detrend_dim(enlarged_ssts, dim="time", deg=1).sel(time=slice("1601","2010"))
y=detrend_dim(ssts_AM_anoms1901[var].fillna(0), dim="time", deg=1).sel(time=slice("1601","2010"))
xr_R,ds7_var,xr_p=corr_with_sig(x,y,sig_level=0.1)

x=detrend_dim(enlarged_ssts, dim="time", deg=1).sel(time=slice("1601","1700"))
y=detrend_dim(ssts_AM_anoms1901[var].fillna(0), dim="time", deg=1).sel(time=slice("1601","1700"))
xr_R,ds8_var,xr_p=corr_with_sig(x,y,sig_level=0.1)

x=detrend_dim(enlarged_ssts, dim="time", deg=1).sel(time=slice("1701","1800"))
y=detrend_dim(ssts_AM_anoms1901[var].fillna(0), dim="time", deg=1).sel(time=slice("1701","1800"))
xr_R,ds4_var,xr_p=corr_with_sig(x,y,sig_level=0.1)

x=detrend_dim(enlarged_ssts, dim="time", deg=1).sel(time=slice("1801","1900"))
y=detrend_dim(ssts_AM_anoms1901[var].fillna(0), dim="time", deg=1).sel(time=slice("1801","1900"))
xr_R,ds5_var,xr_p=corr_with_sig(x,y,sig_level=0.1)

x=detrend_dim(enlarged_ssts, dim="time", deg=1).sel(time=slice("1901","2000"))
y=detrend_dim(ssts_AM_anoms1901[var].fillna(0), dim="time", deg=1).sel(time=slice("1901","2000"))
xr_R,ds6_var,xr_p=corr_with_sig(x,y,sig_level=0.1)




ds_merged=xr.concat([ds7_var,ds8_var,ds4_var,ds5_var,ds6_var],dim="height")





seasons=list(["1601-2010","1601-1700","1701-1800","1801-1900","1901-2000"])
INDEX=list(["(a)","(b)","(c)","(d)","(e)"])
fig, axs = plt.subplots(5, 1, figsize=(6.5, 17), dpi= 300,subplot_kw={'projection': ccrs.EqualEarth(central_longitude=0, globe=None)})
# axs is a 2 dimensional array of `GeoAxes`.  We will flatten it into a 1-D array
axs=axs.flatten()

#Loop over all of the models
for i in range(5):
    tplot=ds_merged[i,:,:].plot.pcolormesh(ax=axs[i],levels = 17, vmax=0.6, transform=ccrs.PlateCarree(), cmap=current_cmap,add_colorbar=False)
    axs[i].set_title(INDEX[i],loc='left')
    axs[i].coastlines()
    axs[i].set_title(seasons[i])

# Adjust the location of the subplots on the page to make room for the colorbar
fig.subplots_adjust(bottom=0.25, top=0.9, left=0.2, right=0.8,
                    wspace=0.1, hspace=0.1)

# Add a colorbar axis at the bottom of the graph
cbar_ax = fig.add_axes([0.2, 0.2, 0.6, 0.02])

# Draw the colorbar
cbar=fig.colorbar(tplot, cax=cbar_ax,orientation='horizontal')
cbar.set_label('Pearson Corr Coeff', size=10) 




# Make it nice
#plt.tight_layout()
fig.savefig(plot_folder+"sst_correlation_all_centuries_01.pdf")

In [ ]:
startyear="1701"
endyear="1800"
vars=list(ssts_AM_anoms1901.keys())
print(vars)
ssts_JJA_corrs_masked=[]
for member in range(len(ssts_member_AM_anoms1901.member)):
    member_str=str(member)
    print(member_str)

    for var in vars:
    
        x=detrend_dim(enlarged_ssts, dim="time", deg=1).sel(time=slice(startyear,endyear))
        y=detrend_dim(ssts_member_AM_anoms1901[var].sel(member=member).fillna(0), dim="time", deg=1).sel(time=slice(startyear,endyear))
        xr_R,xr_R_masked,xr_p=corr_with_sig(x,y,sig_level=0.05)
        ssts_JJA_corrs_masked.append(xr_R_masked)
        
        fig = plt.figure(figsize=(8, 4), dpi= 200)
        ax = plt.axes(projection=ccrs.EqualEarth(central_longitude=0, globe=None)) 
        tplot=xr_R_masked.plot.pcolormesh(ax=ax,
        levels = 17, transform=ccrs.PlateCarree(), vmax=0.6, cmap=current_cmap, cbar_kwargs={'orientation':'vertical',
        'fraction':0.012, 'pad':0.015, 'aspect':35})
    
        tplot.colorbar.set_label('Pearson Corr. Coef.', size=10) 
        tplot.ylabel_style = {'size':16}
        
        ax.set_global()
        ax.coastlines()
        ax.set_title(var+"_"+startyear+endyear)
        #ax.set_extent([-90, 40, 25, 80])
        ;
        fig.savefig(plot_folder+"o18pc1_JJA_CORR_mergedssts_"+var+member_str+"_"+startyear+endyear+".pdf")

# Correlation between SST fields

## era20c

In [ ]:
era20c_1st_may=xr.open_dataset(era20c_folder+"era20c_1stMay_ssts_allyears_daymean.nc")

In [ ]:
era20c_1st_may_anoms1901=create_anomalies_alt(da=era20c_1st_may,month_or_season=5,ref_time=["1901-01-01","2000-12-31"])


In [ ]:
era20c_1st_may_anoms1901_regrid=era20c_1st_may_anoms1901.sst.regrid.linear(fullfield)

In [ ]:
era20c_1st_may_anoms1901_regrid_detrend=detrend_dim(era20c_1st_may_anoms1901_regrid, dim="time", deg=1)


In [ ]:
era20c_1st_may_anoms1901_regrid_detrend_ekf400_corr=xr.corr(era20c_1st_may_anoms1901_regrid_detrend,fullfield,dim=["longitude","latitude"])

In [ ]:
era20c_1st_may_anoms1901_regrid_detrend_ekf400_corr=era20c_1st_may_anoms1901_regrid_detrend_ekf400_corr.sel(time=slice("1901","2010"))

## cera20c

In [ ]:
cera20c_1st_may=xr.open_dataset(cera20c_folder+"cera20c_1stMay_ssts_allyears_daymean.nc")

In [ ]:
cera20c_1st_may_anoms1901=create_anomalies_alt(da=cera20c_1st_may,month_or_season=5,ref_time=["1901-01-01","2000-12-31"])


In [ ]:
cera20c_1st_may_anoms1901_regrid=cera20c_1st_may_anoms1901.sst.regrid.linear(fullfield)

In [ ]:
cera20c_1st_may_anoms1901_regrid_detrend=detrend_dim(cera20c_1st_may_anoms1901_regrid, dim="time", deg=1)


In [ ]:
cera20c_1st_may_anoms1901_regrid_detrend_ekf400_corr=xr.corr(cera20c_1st_may_anoms1901_regrid_detrend,fullfield,dim=["longitude","latitude"])

In [ ]:
cera20c_1st_may_anoms1901_regrid_detrend_ekf400_corr=cera20c_1st_may_anoms1901_regrid_detrend_ekf400_corr.sel(time=slice("1901","2010"))

In [ ]:
corr_CERA20C=pd.DataFrame(cera20c_1st_may_anoms1901_regrid_detrend_ekf400_corr.values, index=pd.period_range("1901-04-16", '2010-4-16',freq="y"),columns=["FLDCOR CERA20C ENSMEAN"])
corr_ERA20C=pd.DataFrame(era20c_1st_may_anoms1901_regrid_detrend_ekf400_corr.values, index=pd.period_range("1901-04-16", '2010-4-16',freq="y"),columns=["FLDCOR ERA20C"])


In [ ]:
quantile_upper_era=corr_ERA20C.quantile(0.75)
quantile_upper_era

In [ ]:
quantile_lower_era=corr_ERA20C.quantile(0.25)
quantile_lower_era

In [ ]:
quantile_upper_cera=corr_CERA20C.quantile(0.75)
quantile_upper_cera

In [ ]:
quantile_lower_cera=corr_CERA20C.quantile(0.25)
quantile_lower_cera

In [ ]:
mask_upper_cera=corr_CERA20C.ge(quantile_upper_cera)
mask_lower_cera=corr_CERA20C.le(quantile_lower_cera)
mask_upper_era=corr_ERA20C.ge(quantile_upper_era)
mask_lower_era=corr_ERA20C.le(quantile_lower_era)


quantile_upper_era_ts=np.repeat(quantile_upper_era, len(corr_ERA20C))
quantile_upper_era_ts=pd.DataFrame(quantile_upper_era_ts.values, index=pd.period_range("1901-04-16", '2010-4-16',freq="y"))

quantile_lower_era_ts=np.repeat(quantile_lower_era, len(corr_ERA20C))
quantile_lower_era_ts=pd.DataFrame(quantile_lower_era_ts.values, index=pd.period_range("1901-04-16", '2010-4-16',freq="y"))

quantile_upper_cera_ts=np.repeat(quantile_upper_cera, len(corr_CERA20C))
quantile_upper_cera_ts=pd.DataFrame(quantile_upper_cera_ts.values, index=pd.period_range("1901-04-16", '2010-4-16',freq="y"))

quantile_lower_cera_ts=np.repeat(quantile_lower_cera, len(corr_CERA20C))
quantile_lower_cera_ts=pd.DataFrame(quantile_lower_cera_ts.values, index=pd.period_range("1901-04-16", '2010-4-16',freq="y"))

In [ ]:
plt.figure(figsize=(10, 5), dpi= 300)
corr_ERA20C.plot(xlabel='Time', ylabel='Pearson Corr Coeff', title='AM vs May 1st SST anomlies field correlation',grid=1,style='b-',colormap="viridis", ax = plt.gca(),ylim=(-1,1), alpha=0.8,label="ERA-20C")
corr_CERA20C.plot(xlabel='Time', ylabel='Pearson Corr Coeff', title='AM vs May 1st SST anomlies field correlation',grid=1,style='g-',colormap="viridis", ax = plt.gca(),ylim=(-1,1), alpha=0.8,label="CERA-20C ensmean")
quantile_upper_era_ts.plot(xlabel='Time', ylabel='Pearson Corr Coeff', title='',grid=1,style='b--', ax = plt.gca(),ylim=(-1,1), alpha=0.2,legend=False)
quantile_lower_era_ts.plot(xlabel='Time', ylabel='Pearson Corr Coeff', title='',grid=1,style='b--', ax = plt.gca(),ylim=(-1,1), alpha=0.2,legend=False)
quantile_upper_cera_ts.plot(xlabel='Time', ylabel='Pearson Corr Coeff', title='J',grid=1,style='g--', ax = plt.gca(),ylim=(-1,1), alpha=0.2,legend=False)
quantile_lower_cera_ts.plot(xlabel='Time', ylabel='Pearson Corr Coeff', title='',grid=1,style='g--', ax = plt.gca(),ylim=(-1,1), alpha=0.2,legend=False)
corr_CERA20C[mask_upper_cera].plot(xlabel='Time', ylabel='Pearson Corr Coeff', title='AM vs May 1st SST anomlies field correlation',grid=1,style='g.',colormap="viridis", ax = plt.gca(),ylim=(-1,1), alpha=0.8,legend=False)
corr_CERA20C[mask_lower_cera].plot(xlabel='Time', ylabel='Pearson Corr Coeff', title='AM vs May 1st SST anomlies field correlation',grid=1,style='g.',colormap="viridis", ax = plt.gca(),ylim=(-1,1), alpha=0.8,legend=False)
corr_ERA20C[mask_upper_era].plot(xlabel='Time', ylabel='Pearson Corr Coeff', title='AM vs May 1st SST anomlies field correlation',grid=1,style='b.',colormap="viridis", ax = plt.gca(),ylim=(-1,1), alpha=0.8,legend=False)
corr_ERA20C[mask_lower_era].plot(xlabel='Time', ylabel='Pearson Corr Coeff', title='AM vs May 1st SST anomlies field correlation',grid=1,style='b.',colormap="viridis", ax = plt.gca(),ylim=(-1,1), alpha=0.8,legend=False)

plt.savefig(plot_folder+"fldcor_SST.pdf")








In [ ]:
corr_CERA20C[mask_upper_cera].dropna().index.to_series().astype(str).values

In [ ]:
corr_CERA20C[mask_lower_cera].dropna().index.to_series().astype(str).values

In [ ]:
corr_ERA20C[mask_upper_era].dropna().index.to_series().astype(str).values

In [ ]:
corr_ERA20C[mask_lower_era].dropna().index.to_series().astype(str).values

# Reading in CSF and ASF and selecting the right years

## csf v250

In [ ]:
#
csf20c_folder_v250=csf20c_folder+"v250/may/"
os.chdir(csf20c_folder_v250)

In [ ]:
!for b in $(seq 0 50); do for a in $(seq -w 1900 2010); do cdo settaxis,${a}-05-15,12:00,1mon V250monthly_${a}_M${b}.nc V250monthly_${a}_M${b}_taxis.nc; done; done
!for b in $(seq 0 50); do cdo -O mergetime V250monthly_*_M${b}_taxis.nc V250monthly_allyears_allseasons_M${b}.nc; done
!for b in $(seq 0 50); do cdo splitmon V250monthly_allyears_allseasons_M${b}.nc V250monthly_allyears_M${b}_; cdo yearmean -selmon,6,7,8 V250monthly_allyears_allseasons_M${b}.nc V250monthly_allyears_M${b}_JJA.nc; done
!for b in $(seq 0 50); do cdo selmon,5 V250monthly_allyears_allseasons_M${b}.nc V250monthly_allyears_M${b}_may.nc; done
!for b in $(seq 0 50); do cdo selmon,6 V250monthly_allyears_allseasons_M${b}.nc V250monthly_allyears_M${b}_jun.nc; done
!for b in $(seq 0 50); do cdo selmon,7 V250monthly_allyears_allseasons_M${b}.nc V250monthly_allyears_M${b}_jul.nc; done
!for b in $(seq 0 50); do cdo selmon,8 V250monthly_allyears_allseasons_M${b}.nc V250monthly_allyears_M${b}_aug.nc; done


In [ ]:
!for b in $(seq 0 51);do cdo -b 32 sellonlatbox,-180,180,-60,90 v250_monthly_allyears_M${b}_JJA.nc v250_monthly_allyears_M${b}_JJA_remapped.nc; done 

In [ ]:
cera20c_v250_JJA=xr.open_mfdataset("v250_monthly_allyears_M*_JJA_remapped.nc", combine='nested',concat_dim="plev")

In [ ]:
year_list=[]
for i in corr_CERA20C[mask_lower_cera].dropna().index.to_series().astype(str).values:
    year_list.append(i)
cera20c_v250_JJA_negcorrs=cera20c_v250_JJA.sel(time=year_list,method="bfill")

In [ ]:
year_list=[]
for i in corr_CERA20C[mask_upper_cera].dropna().index.to_series().astype(str).values:
    year_list.append(i)
cera20c_v250_JJA_poscorrs=cera20c_v250_JJA.sel(time=year_list,method="bfill")

In [ ]:
t, p = scipy.stats.ttest_ind(cera20c_v250_JJA_poscorrs.mean(dim="plev").v, cera20c_v250_JJA_negcorrs.mean(dim="plev").v)
anomalies_mean=cera20c_v250_JJA_poscorrs.mean(dim="time").v-cera20c_v250_JJA_negcorrs.mean(dim="time").v


fig = plt.figure(figsize=(8, 4), dpi= 200)
ax = plt.axes(projection=ccrs.EqualEarth(central_longitude=0, globe=None)) 
tplot=anomalies_mean.mean(dim="plev").where(p<0.05).plot.pcolormesh(ax=ax,
levels = 17, transform=ccrs.PlateCarree(), cmap=current_cmap_v, vmax=1.6,cbar_kwargs={'orientation':'vertical',
'fraction':0.012, 'pad':0.015, 'aspect':35})

tplot.colorbar.set_label('Pearson Corr. Coef.', size=10) 
tplot.ylabel_style = {'size':16}

tplot.colorbar.set_label("[m/s]", size=10) 
tplot.ylabel_style = {'size':16}

ax.set_global()
ax.coastlines()
ax.set_title('CSF20C')
ax.set_title("(b)",loc='left')
#ax.set_extent([-179, 179, -60, 70])
ax.set_extent([-90, 50, 25, 80])
;
plt.savefig(plot_folder+"v250_JJA_csf20c.pdf")

## csf t2m

In [ ]:
#
csf20c_folder_t2m=csf20c_folder+"t2m/may/"
os.chdir(csf20c_folder_t2m)

In [ ]:
!for b in $(seq 0 51);do cdo -b 32 sellonlatbox,-180,180,-60,90 t2mmonthly_allyears_M${b}_JJA.nc t2m_monthly_allyears_M${b}_JJA_remapped.nc; done 

In [ ]:
cera20c_t2m_JJA=xr.open_mfdataset("t2m_monthly_allyears_M*_JJA_remapped.nc", combine='nested',concat_dim="plev")

In [ ]:
year_list=[]
for i in corr_CERA20C[mask_lower_cera].dropna().index.to_series().astype(str).values:
    year_list.append(i)
cera20c_t2m_JJA_negcorrs=cera20c_t2m_JJA.sel(time=year_list,method="bfill")

In [ ]:
year_list=[]
for i in corr_CERA20C[mask_upper_cera].dropna().index.to_series().astype(str).values:
    year_list.append(i)
cera20c_t2m_JJA_poscorrs=cera20c_t2m_JJA.sel(time=year_list,method="bfill")

In [ ]:
t, p = scipy.stats.ttest_ind(cera20c_t2m_JJA_poscorrs.mean(dim="plev")["2t"], cera20c_t2m_JJA_negcorrs.mean(dim="plev")["2t"])
anomalies_mean=cera20c_t2m_JJA_poscorrs.mean(dim="time")["2t"]-cera20c_t2m_JJA_negcorrs.mean(dim="time")["2t"]


fig = plt.figure(figsize=(8, 4), dpi= 200)
ax = plt.axes(projection=ccrs.EqualEarth(central_longitude=0, globe=None)) 
tplot=anomalies_mean.mean(dim="plev").where(p<0.05).plot.pcolormesh(ax=ax,
levels = 17, transform=ccrs.PlateCarree(), cmap=current_cmap_t, vmax=1.6,cbar_kwargs={'orientation':'vertical',
'fraction':0.012, 'pad':0.015, 'aspect':35})

tplot.colorbar.set_label('Pearson Corr. Coef.', size=10) 
tplot.ylabel_style = {'size':16}

tplot.colorbar.set_label("[K]", size=10) 
tplot.ylabel_style = {'size':16}

ax.set_global()
ax.coastlines()
ax.set_title('CSF20C')
ax.set_title("(d)",loc='left')
#ax.set_extent([-179, 179, -60, 70])
ax.set_extent([-90, 50, 25, 80])
;
;
plt.savefig(plot_folder+"t2m_JJA_csf20c.pdf")

## csf precip

In [ ]:
#
csf20c_folder_precip=csf20c_folder+"precip/may/"
os.chdir(csf20c_folder_precip)

In [ ]:
!for b in $(seq 0 51);do cdo -b 32 sellonlatbox,-180,180,-60,90 tpratemonthly_allyears_M${b}_JJA.nc tprate_monthly_allyears_M${b}_JJA_remapped.nc; done 

In [ ]:
cera20c_tprate_JJA=xr.open_mfdataset("tprate_monthly_allyears_M*_JJA_remapped.nc", combine='nested',concat_dim="plev")

In [ ]:
year_list=[]
for i in corr_CERA20C[mask_lower_cera].dropna().index.to_series().astype(str).values:
    year_list.append(i)
cera20c_tprate_JJA_negcorrs=cera20c_tprate_JJA.sel(time=year_list,method="bfill")

In [ ]:
year_list=[]
for i in corr_CERA20C[mask_upper_cera].dropna().index.to_series().astype(str).values:
    year_list.append(i)
cera20c_tprate_JJA_poscorrs=cera20c_tprate_JJA.sel(time=year_list,method="bfill")

In [ ]:
t, p = scipy.stats.ttest_ind(cera20c_tprate_JJA_poscorrs.mean(dim="plev")["tprate"], cera20c_tprate_JJA_negcorrs.mean(dim="plev")["tprate"])
anomalies_mean=cera20c_tprate_JJA_poscorrs.mean(dim="time")["tprate"]-cera20c_tprate_JJA_negcorrs.mean(dim="time")["tprate"]

anomalies_mean=anomalies_mean*1000*86400
fig = plt.figure(figsize=(8, 4), dpi= 200)
ax = plt.axes(projection=ccrs.EqualEarth(central_longitude=0, globe=None)) 
tplot=anomalies_mean.mean(dim="plev").where(p<0.05).plot.pcolormesh(ax=ax,
levels = 17, transform=ccrs.PlateCarree(), cmap=current_cmap_rain, vmax=0.8,cbar_kwargs={'orientation':'vertical',
'fraction':0.012, 'pad':0.015, 'aspect':35})

tplot.colorbar.set_label('Pearson Corr. Coef.', size=10) 
tplot.ylabel_style = {'size':16}

tplot.colorbar.set_label("[mm/day]", size=10) 
tplot.ylabel_style = {'size':16}

ax.set_global()
ax.coastlines()
ax.set_title('CSF20C')
ax.set_title("(f)",loc='left')
#ax.set_extent([-179, 179, -60, 70])
ax.set_extent([-90, 50, 25, 80])
;
plt.savefig(plot_folder+"tprate_JJA_csf20c.pdf")

## asf v250

In [ ]:
#
asf20c_folder_v250=asf20c_folder+"v250/may/"
os.chdir(asf20c_folder_v250)

In [ ]:
!for b in $(seq 0 50); do for a in $(seq -w 1900 2010); do cdo settaxis,${a}-05-15,12:00,1mon V250monthly_${a}_M${b}.nc V250monthly_${a}_M${b}_taxis.nc; done; done
!for b in $(seq 0 50); do cdo -O mergetime V250monthly_*_M${b}_taxis.nc V250monthly_allyears_allseasons_M${b}.nc; done
!for b in $(seq 0 50); do cdo splitmon V250monthly_allyears_allseasons_M${b}.nc V250monthly_allyears_M${b}_; cdo yearmean -selmon,6,7,8 V250monthly_allyears_allseasons_M${b}.nc V250monthly_allyears_M${b}_JJA.nc; done
!for b in $(seq 0 50); do cdo selmon,5 V250monthly_allyears_allseasons_M${b}.nc V250monthly_allyears_M${b}_may.nc; done
!for b in $(seq 0 50); do cdo selmon,6 V250monthly_allyears_allseasons_M${b}.nc V250monthly_allyears_M${b}_jun.nc; done
!for b in $(seq 0 50); do cdo selmon,7 V250monthly_allyears_allseasons_M${b}.nc V250monthly_allyears_M${b}_jul.nc; done
!for b in $(seq 0 50); do cdo selmon,8 V250monthly_allyears_allseasons_M${b}.nc V250monthly_allyears_M${b}_aug.nc; done


In [ ]:
!for b in $(seq 0 51);do cdo -b 32 sellonlatbox,-180,180,-60,90 v250_monthly_allyears_M${b}_JJA.nc v250_monthly_allyears_M${b}_JJA_remapped.nc; done 

In [ ]:
era20c_v250_JJA=xr.open_mfdataset("v250_monthly_allyears_M*_JJA_remapped.nc", combine='nested',concat_dim="plev")

In [ ]:
year_list=[]
for i in corr_ERA20C[mask_lower_era].dropna().index.to_series().astype(str).values:
    year_list.append(i)
era20c_v250_JJA_negcorrs=era20c_v250_JJA.sel(time=year_list,method="bfill")

In [ ]:
year_list=[]
for i in corr_ERA20C[mask_upper_era].dropna().index.to_series().astype(str).values:
    year_list.append(i)
era20c_v250_JJA_poscorrs=era20c_v250_JJA.sel(time=year_list,method="bfill")

In [ ]:
t, p = scipy.stats.ttest_ind(era20c_v250_JJA_poscorrs.mean(dim="plev").v, era20c_v250_JJA_negcorrs.mean(dim="plev").v)
anomalies_mean=era20c_v250_JJA_poscorrs.mean(dim="time").v-era20c_v250_JJA_negcorrs.mean(dim="time").v


fig = plt.figure(figsize=(8, 4), dpi= 200)
ax = plt.axes(projection=ccrs.EqualEarth(central_longitude=0, globe=None)) 
tplot=anomalies_mean.mean(dim="plev").where(p<0.05).plot.pcolormesh(ax=ax,
levels = 17, transform=ccrs.PlateCarree(), cmap=current_cmap_v, vmax=1.6,cbar_kwargs={'orientation':'vertical',
'fraction':0.012, 'pad':0.015, 'aspect':35})

tplot.colorbar.set_label('Pearson Corr. Coef.', size=10) 
tplot.ylabel_style = {'size':16}
 
tplot.colorbar.set_label("[m/s]", size=10) 
tplot.ylabel_style = {'size':16}

ax.set_global()
ax.coastlines()
ax.set_title('ASF20C')
ax.set_title("(a)",loc='left')
#ax.set_extent([-179, 179, -60, 70])
ax.set_extent([-90, 50, 25, 80])
;
plt.savefig(plot_folder+"v250_JJA_asf20c.pdf")

## asf t2m

In [ ]:
#
asf20c_folder_t2m=asf20c_folder+"t2m/may/"
os.chdir(asf20c_folder_t2m)

In [ ]:
!for b in $(seq 0 51);do cdo -b 32 sellonlatbox,-180,180,-60,90 t2mmonthly_allyears_M${b}_JJA.nc t2m_monthly_allyears_M${b}_JJA_remapped.nc; done 

In [ ]:
era20c_t2m_JJA=xr.open_mfdataset("t2m_monthly_allyears_M*_JJA_remapped.nc", combine='nested',concat_dim="plev")

In [ ]:
year_list=[]
for i in corr_ERA20C[mask_lower_era].dropna().index.to_series().astype(str).values:
    year_list.append(i)
era20c_t2m_JJA_negcorrs=era20c_t2m_JJA.sel(time=year_list,method="bfill")

In [ ]:
year_list=[]
for i in corr_ERA20C[mask_upper_era].dropna().index.to_series().astype(str).values:
    year_list.append(i)
era20c_t2m_JJA_poscorrs=era20c_t2m_JJA.sel(time=year_list,method="bfill")

In [ ]:
t, p = scipy.stats.ttest_ind(era20c_t2m_JJA_poscorrs.mean(dim="plev")["2t"], era20c_t2m_JJA_negcorrs.mean(dim="plev")["2t"])
anomalies_mean=era20c_t2m_JJA_poscorrs.mean(dim="time")["2t"]-era20c_t2m_JJA_negcorrs.mean(dim="time")["2t"]


fig = plt.figure(figsize=(8, 4), dpi= 200)
ax = plt.axes(projection=ccrs.EqualEarth(central_longitude=0, globe=None)) 
tplot=anomalies_mean.mean(dim="plev").where(p<0.05).plot.pcolormesh(ax=ax,
levels = 17, transform=ccrs.PlateCarree(), cmap=current_cmap_t, vmax=1.6,cbar_kwargs={'orientation':'vertical',
'fraction':0.012, 'pad':0.015, 'aspect':35})

tplot.colorbar.set_label('Pearson Corr. Coef.', size=10) 
tplot.ylabel_style = {'size':16}

tplot.colorbar.set_label("[K]", size=10) 
tplot.ylabel_style = {'size':16}

ax.set_global()
ax.coastlines()
ax.set_title('CSF20C')
ax.set_title("(c)",loc='left')
#ax.set_extent([-179, 179, -60, 70])
ax.set_extent([-90, 50, 25, 80])
;
plt.savefig(plot_folder+"t2m_JJA_asf20c.pdf")

## asf precip

In [ ]:
#
asf20c_folder_precip=asf20c_folder+"precip/may/"
os.chdir(asf20c_folder_precip)

In [ ]:
!for b in $(seq 0 52);do cdo -b 32 sellonlatbox,-180,180,-60,90 -yearmean -selmon,7,8,9 afs20_ctl_allyears_6789_${b}_tprate.nc tprate_monthly_allyears_M${b}_JJA_remapped.nc; done 

In [ ]:
era20c_tprate_JJA=xr.open_mfdataset("tprate_monthly_allyears_M*_JJA_remapped.nc", combine='nested',concat_dim="plev")

In [ ]:
year_list=[]
for i in corr_ERA20C[mask_lower_era].dropna().index.to_series().astype(str).values:
    year_list.append(i)
era20c_tprate_JJA_negcorrs=era20c_tprate_JJA.sel(time=year_list,method="bfill")

In [ ]:
year_list=[]
for i in corr_ERA20C[mask_upper_era].dropna().index.to_series().astype(str).values:
    year_list.append(i)
era20c_tprate_JJA_poscorrs=era20c_tprate_JJA.sel(time=year_list,method="bfill")

In [ ]:
t, p = scipy.stats.ttest_ind(era20c_tprate_JJA_poscorrs.mean(dim="plev")["tprate"], era20c_tprate_JJA_negcorrs.mean(dim="plev")["tprate"])
anomalies_mean=era20c_tprate_JJA_poscorrs.mean(dim="time")["tprate"]-cera20c_tprate_JJA_negcorrs.mean(dim="time")["tprate"]
anomalies_mean=anomalies_mean*1000*86400


fig = plt.figure(figsize=(8, 4), dpi= 200)
ax = plt.axes(projection=ccrs.EqualEarth(central_longitude=0, globe=None)) 
tplot=anomalies_mean.mean(dim="plev").where(p<0.05).plot.pcolormesh(ax=ax,
levels = 17, transform=ccrs.PlateCarree(), cmap=current_cmap_rain, vmax=0.8,cbar_kwargs={'orientation':'vertical',
'fraction':0.012, 'pad':0.015, 'aspect':35})

tplot.colorbar.set_label('Pearson Corr. Coef.', size=10) 
tplot.ylabel_style = {'size':16}

tplot.colorbar.set_label("[mm/day]", size=10) 
tplot.ylabel_style = {'size':16}

ax.set_global()
ax.coastlines()
ax.set_title('ASF20C')
ax.set_title("(e)",loc='left')
#ax.set_extent([-179, 179, -60, 70])
ax.set_extent([-90, 50, 25, 80])
;
plt.savefig(plot_folder+"tprate_JJA_asf20c.pdf")

# atmospheric development in csf20c

## OLR

### May

In [ ]:
#
csf_folder_olr=csf20c_folder+"olr/may/"
os.chdir(csf_folder_olr)

In [ ]:
!for b in $(seq 0 51);do cdo -b 32 sellonlatbox,-180,180,-60,90 var179monthly_allyears_M${b}_may.nc var179monthly_allyears_M${b}_may_remapped.nc; done 

In [ ]:
cera20c_olr_may=xr.open_mfdataset("var179monthly_allyears_M*_may_remapped.nc", combine='nested',concat_dim="plev")

In [ ]:
#cera20c_olr_may=cera20c_olr_may.rename({'forecast_time0': 'time'})
#cera20c_olr_may=cera20c_olr_may.rename({'VAR_179_GDS0_SFC': 'olr'})
#cera20c_olr_may=cera20c_olr_may.rename({'g0_lat_1': 'lat'})
#cera20c_olr_may=cera20c_olr_may.rename({'g0_lat_1': 'lat'})

In [ ]:
year_list=[]
for i in corr_CERA20C[mask_lower_cera].dropna().index.to_series().astype(str).values:
    year_list.append(i)
cera20c_olr_may_negcorrs=cera20c_olr_may.sel(time=year_list,method="bfill")

In [ ]:
year_list=[]
for i in corr_CERA20C[mask_upper_cera].dropna().index.to_series().astype(str).values:
    year_list.append(i)
cera20c_olr_may_poscorrs=cera20c_olr_may.sel(time=year_list,method="bfill")

In [ ]:
t, p = scipy.stats.ttest_ind(cera20c_olr_may_poscorrs.mean(dim="plev").var179, cera20c_olr_may_negcorrs.mean(dim="plev").var179)
anomalies_mean=cera20c_olr_may_poscorrs.mean(dim="time").var179-cera20c_olr_may_negcorrs.mean(dim="time").var179


fig = plt.figure(figsize=(8, 4), dpi= 200)
ax = plt.axes(projection=ccrs.EqualEarth(central_longitude=0, globe=None)) 
tplot=anomalies_mean.mean(dim="plev").where(p<0.05).plot.pcolormesh(ax=ax,
levels = 17, transform=ccrs.PlateCarree(), vmax=40,cmap=current_cmap_olr,cbar_kwargs={'orientation':'horizontal',
'fraction':0.12, 'pad':0.015, 'aspect':35})

tplot.colorbar.set_label('Pearson Corr. Coef.', size=10) 
tplot.ylabel_style = {'size':16}

tplot.colorbar.set_label("[W/m^2]", size=10) 
tplot.ylabel_style = {'size':16}

ax.set_global()
ax.coastlines()
ax.set_title('OLR')
ax.set_title("(a)",loc='left')
ax.set_extent([-179, 179, -60, 70])
#ax.set_extent([-90, 50, 25, 80])
;
plt.savefig(plot_folder+"olr_may_csf20c.pdf")

### june

In [ ]:
!for b in $(seq 0 51);do cdo -b 32 sellonlatbox,-180,180,-60,90 var179monthly_allyears_M${b}_jun.nc var179monthly_allyears_M${b}_jun_remapped.nc; done 

In [ ]:
cera20c_olr_jun=xr.open_mfdataset("var179monthly_allyears_M*_jun_remapped.nc", combine='nested',concat_dim="plev")

In [ ]:
year_list=[]
for i in corr_CERA20C[mask_lower_cera].dropna().index.to_series().astype(str).values:
    year_list.append(i)
cera20c_olr_jun_negcorrs=cera20c_olr_jun.sel(time=year_list,method="bfill")

In [ ]:
year_list=[]
for i in corr_CERA20C[mask_upper_cera].dropna().index.to_series().astype(str).values:
    year_list.append(i)
cera20c_olr_jun_poscorrs=cera20c_olr_jun.sel(time=year_list,method="bfill")

In [ ]:
t, p = scipy.stats.ttest_ind(cera20c_olr_jun_poscorrs.mean(dim="plev").var179, cera20c_olr_jun_negcorrs.mean(dim="plev").var179)
anomalies_mean=cera20c_olr_jun_poscorrs.mean(dim="time").var179-cera20c_olr_jun_negcorrs.mean(dim="time").var179


fig = plt.figure(figsize=(8, 4), dpi= 200)
ax = plt.axes(projection=ccrs.EqualEarth(central_longitude=0, globe=None)) 
tplot=anomalies_mean.mean(dim="plev").where(p<0.05).plot.pcolormesh(ax=ax,
levels = 17, transform=ccrs.PlateCarree(), vmax=40,cmap=current_cmap_olr,cbar_kwargs={'orientation':'horizontal',
'fraction':0.12, 'pad':0.015, 'aspect':35})

tplot.colorbar.set_label('Pearson Corr. Coef.', size=10) 
tplot.ylabel_style = {'size':16}

tplot.colorbar.set_label("[W/m^2]", size=10) 
tplot.ylabel_style = {'size':16}

ax.set_global()
ax.coastlines()
ax.set_title('OLR')
ax.set_title("(d)",loc='left')
ax.set_extent([-179, 179, -60, 70])
#ax.set_extent([-90, 50, 25, 80])
;
;
plt.savefig(plot_folder+"olr_jun_csf20c.pdf")

### july

In [ ]:
!for b in $(seq 0 51);do cdo -b 32 sellonlatbox,-180,180,-60,90 var179monthly_allyears_M${b}_jul.nc var179monthly_allyears_M${b}_jul_remapped.nc; done 

In [ ]:
cera20c_olr_jul=xr.open_mfdataset("var179monthly_allyears_M*_jul_remapped.nc", combine='nested',concat_dim="plev")

In [ ]:
year_list=[]
for i in corr_CERA20C[mask_lower_cera].dropna().index.to_series().astype(str).values:
    year_list.append(i)
cera20c_olr_jul_negcorrs=cera20c_olr_jul.sel(time=year_list,method="bfill")

In [ ]:
year_list=[]
for i in corr_CERA20C[mask_upper_cera].dropna().index.to_series().astype(str).values:
    year_list.append(i)
cera20c_olr_jul_poscorrs=cera20c_olr_jul.sel(time=year_list,method="bfill")

In [ ]:
t, p = scipy.stats.ttest_ind(cera20c_olr_jul_poscorrs.mean(dim="plev").var179, cera20c_olr_jul_negcorrs.mean(dim="plev").var179)
anomalies_mean=cera20c_olr_jul_poscorrs.mean(dim="time").var179-cera20c_olr_jul_negcorrs.mean(dim="time").var179


fig = plt.figure(figsize=(8, 4), dpi= 200)
ax = plt.axes(projection=ccrs.EqualEarth(central_longitude=0, globe=None)) 
tplot=anomalies_mean.mean(dim="plev").where(p<0.05).plot.pcolormesh(ax=ax,
levels = 17, transform=ccrs.PlateCarree(), vmax=40,cmap=current_cmap_olr,cbar_kwargs={'orientation':'horizontal',
'fraction':0.12, 'pad':0.015, 'aspect':35})

tplot.colorbar.set_label('Pearson Corr. Coef.', size=10) 
tplot.ylabel_style = {'size':16}

tplot.colorbar.set_label("[W/m^2]", size=10) 
tplot.ylabel_style = {'size':16}

ax.set_global()
ax.coastlines()
ax.set_title('OLR')
ax.set_title("(g)",loc='left')
ax.set_extent([-179, 179, -60, 70])
#ax.set_extent([-90, 50, 25, 80])
;
;
plt.savefig(plot_folder+"olr_jul_csf20c.pdf")

## V250

### May

In [ ]:
#
csf20c_folder_v250=csf20c_folder+"v250/may/"
os.chdir(csf20c_folder_v250)

In [ ]:
!for b in $(seq 0 51);do cdo -b 32 sellonlatbox,-180,180,-60,90 v250_monthly_allyears_M${b}_may.nc v250_monthly_allyears_M${b}_may_remapped.nc; done 

In [ ]:
cera20c_v250_may=xr.open_mfdataset("v250_monthly_allyears_M*_may_remapped.nc", combine='nested',concat_dim="plev")

In [ ]:
year_list=[]
for i in corr_CERA20C[mask_lower_cera].dropna().index.to_series().astype(str).values:
    year_list.append(i)
cera20c_v250_may_negcorrs=cera20c_v250_may.sel(time=year_list,method="bfill")

In [ ]:
year_list=[]
for i in corr_CERA20C[mask_upper_cera].dropna().index.to_series().astype(str).values:
    year_list.append(i)
cera20c_v250_may_poscorrs=cera20c_v250_may.sel(time=year_list,method="bfill")

In [ ]:
t, p = scipy.stats.ttest_ind(cera20c_v250_may_poscorrs.mean(dim="plev").v, cera20c_v250_may_negcorrs.mean(dim="plev").v)
anomalies_mean=cera20c_v250_may_poscorrs.mean(dim="time").v-cera20c_v250_may_negcorrs.mean(dim="time").v


fig = plt.figure(figsize=(8, 4), dpi= 200)
ax = plt.axes(projection=ccrs.EqualEarth(central_longitude=0, globe=None)) 
tplot=anomalies_mean.mean(dim="plev").where(p<0.05).plot.pcolormesh(ax=ax,
levels = 17, transform=ccrs.PlateCarree(), vmax=5,cmap=current_cmap_v,cbar_kwargs={'orientation':'horizontal',
'fraction':0.12, 'pad':0.015, 'aspect':35})

tplot.colorbar.set_label('Pearson Corr. Coef.', size=10) 
tplot.ylabel_style = {'size':16}

tplot.colorbar.set_label("[m/s]", size=10) 
tplot.ylabel_style = {'size':16}

ax.set_global()
ax.coastlines()
ax.set_title('V250')
ax.set_title("(b)",loc='left')
ax.set_extent([-179, 179, -60, 70])
#ax.set_extent([-90, 50, 25, 80])
;
;
plt.savefig(plot_folder+"v250_may_csf20c.pdf")

### June

In [ ]:
!for b in $(seq 0 51);do cdo -b 32 sellonlatbox,-180,180,-60,90 v250_monthly_allyears_M${b}_jun.nc v250_monthly_allyears_M${b}_jun_remapped.nc; done 

In [ ]:
cera20c_v250_jun=xr.open_mfdataset("v250_monthly_allyears_M*_jun_remapped.nc", combine='nested',concat_dim="plev")

In [ ]:
year_list=[]
for i in corr_CERA20C[mask_lower_cera].dropna().index.to_series().astype(str).values:
    year_list.append(i)
cera20c_v250_jun_negcorrs=cera20c_v250_jun.sel(time=year_list,method="bfill")

In [ ]:
year_list=[]
for i in corr_CERA20C[mask_upper_cera].dropna().index.to_series().astype(str).values:
    year_list.append(i)
cera20c_v250_jun_poscorrs=cera20c_v250_jun.sel(time=year_list,method="bfill")

In [ ]:
t, p = scipy.stats.ttest_ind(cera20c_v250_jun_poscorrs.mean(dim="plev").v, cera20c_v250_jun_negcorrs.mean(dim="plev").v)
anomalies_mean=cera20c_v250_jun_poscorrs.mean(dim="time").v-cera20c_v250_jun_negcorrs.mean(dim="time").v


fig = plt.figure(figsize=(8, 4), dpi= 200)
ax = plt.axes(projection=ccrs.EqualEarth(central_longitude=0, globe=None)) 
tplot=anomalies_mean.mean(dim="plev").where(p<0.05).plot.pcolormesh(ax=ax,
levels = 17, transform=ccrs.PlateCarree(), vmax=5,cmap=current_cmap_v,cbar_kwargs={'orientation':'horizontal',
'fraction':0.12, 'pad':0.015, 'aspect':35})

tplot.colorbar.set_label('Pearson Corr. Coef.', size=10) 
tplot.ylabel_style = {'size':16}

tplot.colorbar.set_label("[m/s]", size=10) 
tplot.ylabel_style = {'size':16}

ax.set_global()
ax.coastlines()
ax.set_title('V250')
ax.set_title("(e)",loc='left')
ax.set_extent([-179, 179, -60, 70])
#ax.set_extent([-90, 50, 25, 80])
;
plt.savefig(plot_folder+"v250_jun_csf20c.pdf")

### July

In [ ]:
!for b in $(seq 0 51);do cdo -b 32 sellonlatbox,-180,180,-60,90 v250_monthly_allyears_M${b}_jul.nc v250_monthly_allyears_M${b}_jul_remapped.nc; done 

In [ ]:
cera20c_v250_jul=xr.open_mfdataset("v250_monthly_allyears_M*_jul_remapped.nc", combine='nested',concat_dim="plev")

In [ ]:
year_list=[]
for i in corr_CERA20C[mask_lower_cera].dropna().index.to_series().astype(str).values:
    year_list.append(i)
cera20c_v250_jul_negcorrs=cera20c_v250_jul.sel(time=year_list,method="bfill")

In [ ]:
year_list=[]
for i in corr_CERA20C[mask_upper_cera].dropna().index.to_series().astype(str).values:
    year_list.append(i)
cera20c_v250_jul_poscorrs=cera20c_v250_jul.sel(time=year_list,method="bfill")

In [ ]:
t, p = scipy.stats.ttest_ind(cera20c_v250_jul_poscorrs.mean(dim="plev").v, cera20c_v250_jul_negcorrs.mean(dim="plev").v)
anomalies_mean=cera20c_v250_jul_poscorrs.mean(dim="time").v-cera20c_v250_jul_negcorrs.mean(dim="time").v


fig = plt.figure(figsize=(8, 4), dpi= 200)
ax = plt.axes(projection=ccrs.EqualEarth(central_longitude=0, globe=None)) 
tplot=anomalies_mean.mean(dim="plev").where(p<0.05).plot.pcolormesh(ax=ax,
levels = 17, transform=ccrs.PlateCarree(), vmax=5,cmap=current_cmap_v,cbar_kwargs={'orientation':'horizontal',
'fraction':0.12, 'pad':0.015, 'aspect':35})

tplot.colorbar.set_label('Pearson Corr. Coef.', size=10) 
tplot.ylabel_style = {'size':16}

tplot.colorbar.set_label("[m/s]", size=10) 
tplot.ylabel_style = {'size':16}

ax.set_global()
ax.coastlines()
ax.set_title('V250')
ax.set_title("(h)",loc='left')
ax.set_extent([-179, 179, -60, 70])
#ax.set_extent([-90, 50, 25, 80])
;
plt.savefig(plot_folder+"v250_jul_csf20c.pdf")

## Z500

### May

In [ ]:
#
csf20c_folder_z500=csf20c_folder+"z500/may/"
os.chdir(csf20c_folder_z500)

In [ ]:
!for b in $(seq 0 51);do cdo -b 32 sellonlatbox,-180,180,-60,90 z500_monthly_allyears_M${b}_may.nc z500_monthly_allyears_M${b}_may_remapped.nc; done 

In [ ]:
cera20c_z500_may=xr.open_mfdataset("z500_monthly_allyears_M*_may_remapped.nc", combine='nested',concat_dim="plev")

In [ ]:
year_list=[]
for i in corr_CERA20C[mask_lower_cera].dropna().index.to_series().astype(str).values:
    year_list.append(i)
cera20c_z500_may_negcorrs=cera20c_z500_may.sel(time=year_list,method="bfill")

In [ ]:
year_list=[]
for i in corr_CERA20C[mask_upper_cera].dropna().index.to_series().astype(str).values:
    year_list.append(i)
cera20c_z500_may_poscorrs=cera20c_z500_may.sel(time=year_list,method="bfill")

In [ ]:
t, p = scipy.stats.ttest_ind(cera20c_z500_may_poscorrs.mean(dim="plev").z, cera20c_z500_may_negcorrs.mean(dim="plev").z)
anomalies_mean=cera20c_z500_may_poscorrs.mean(dim="time").z-cera20c_z500_may_negcorrs.mean(dim="time").z


fig = plt.figure(figsize=(8, 4), dpi= 200)
ax = plt.axes(projection=ccrs.EqualEarth(central_longitude=0, globe=None)) 
tplot=anomalies_mean.mean(dim="plev").where(p<0.05).plot.pcolormesh(ax=ax,
levels = 17, transform=ccrs.PlateCarree(), vmax=500,cmap=current_cmap_z,cbar_kwargs={'orientation':'horizontal',
'fraction':0.12, 'pad':0.015, 'aspect':35})

tplot.colorbar.set_label('Pearson Corr. Coef.', size=10) 
tplot.ylabel_style = {'size':16}

tplot.colorbar.set_label("[Pa]", size=10) 
tplot.ylabel_style = {'size':16}

ax.set_global()
ax.coastlines()
ax.set_title('Z500')
ax.set_title("(c)",loc='left')
ax.set_extent([-179, 179, -60, 70])
#ax.set_extent([-90, 50, 25, 80])
;
plt.savefig(plot_folder+"z500_may_csf20c.pdf")

### June

In [ ]:
!for b in $(seq 0 51);do cdo -b 32 sellonlatbox,-180,180,-60,90 z500_monthly_allyears_M${b}_jun.nc z500_monthly_allyears_M${b}_jun_remapped.nc; done 

In [ ]:
cera20c_z500_jun=xr.open_mfdataset("z500_monthly_allyears_M*_jun_remapped.nc", combine='nested',concat_dim="plev")

In [ ]:
year_list=[]
for i in corr_CERA20C[mask_lower_cera].dropna().index.to_series().astype(str).values:
    year_list.append(i)
cera20c_z500_jun_negcorrs=cera20c_z500_jun.sel(time=year_list,method="bfill")

In [ ]:
year_list=[]
for i in corr_CERA20C[mask_upper_cera].dropna().index.to_series().astype(str).values:
    year_list.append(i)
cera20c_z500_jun_poscorrs=cera20c_z500_jun.sel(time=year_list,method="bfill")

In [ ]:
t, p = scipy.stats.ttest_ind(cera20c_z500_jun_poscorrs.mean(dim="plev").z, cera20c_z500_jun_negcorrs.mean(dim="plev").z)
anomalies_mean=cera20c_z500_jun_poscorrs.mean(dim="time").z-cera20c_z500_jun_negcorrs.mean(dim="time").z


fig = plt.figure(figsize=(8, 4), dpi= 200)
ax = plt.axes(projection=ccrs.EqualEarth(central_longitude=0, globe=None)) 
tplot=anomalies_mean.mean(dim="plev").where(p<0.05).plot.pcolormesh(ax=ax,
levels = 17, transform=ccrs.PlateCarree(), vmax=500,cmap=current_cmap_z,cbar_kwargs={'orientation':'horizontal',
'fraction':0.12, 'pad':0.015, 'aspect':35})

tplot.colorbar.set_label('Pearson Corr. Coef.', size=10) 
tplot.ylabel_style = {'size':16}

tplot.colorbar.set_label("[Pa]", size=10) 
tplot.ylabel_style = {'size':16}

ax.set_global()
ax.coastlines()
ax.set_title('Z500')
ax.set_title("(f)",loc='left')
ax.set_extent([-179, 179, -60, 70])
#ax.set_extent([-90, 50, 25, 80])
;
plt.savefig(plot_folder+"z500_jun_csf20c.pdf")

### July

In [ ]:
!for b in $(seq 0 51);do cdo -b 32 sellonlatbox,-180,180,-60,90 z500_monthly_allyears_M${b}_jul.nc z500_monthly_allyears_M${b}_jul_remapped.nc; done 

In [ ]:
cera20c_z500_jul=xr.open_mfdataset("z500_monthly_allyears_M*_jul_remapped.nc", combine='nested',concat_dim="plev")

In [ ]:
year_list=[]
for i in corr_CERA20C[mask_lower_cera].dropna().index.to_series().astype(str).values:
    year_list.append(i)
cera20c_z500_jul_negcorrs=cera20c_z500_jul.sel(time=year_list,method="bfill")

In [ ]:
year_list=[]
for i in corr_CERA20C[mask_upper_cera].dropna().index.to_series().astype(str).values:
    year_list.append(i)
cera20c_z500_jul_poscorrs=cera20c_z500_jul.sel(time=year_list,method="bfill")

In [ ]:
t, p = scipy.stats.ttest_ind(cera20c_z500_jul_poscorrs.mean(dim="plev").z, cera20c_z500_jul_negcorrs.mean(dim="plev").z)
anomalies_mean=cera20c_z500_jul_poscorrs.mean(dim="time").z-cera20c_z500_jul_negcorrs.mean(dim="time").z

fig = plt.figure(figsize=(8, 4), dpi= 200)
ax = plt.axes(projection=ccrs.EqualEarth(central_longitude=0, globe=None)) 
tplot=anomalies_mean.mean(dim="plev").where(p<0.05).plot.pcolormesh(ax=ax,
levels = 17, transform=ccrs.PlateCarree(), vmax=500,cmap=current_cmap_z,cbar_kwargs={'orientation':'horizontal',
'fraction':0.12, 'pad':0.015, 'aspect':35})

tplot.colorbar.set_label('Pearson Corr. Coef.', size=10) 
tplot.ylabel_style = {'size':16}

tplot.colorbar.set_label("[Pa]", size=10) 
tplot.ylabel_style = {'size':16}

ax.set_global()
ax.coastlines()
ax.set_title('Z500')
ax.set_title("(i)",loc='left')
ax.set_extent([-179, 179, -60, 70])
#ax.set_extent([-90, 50, 25, 80])
;
plt.savefig(plot_folder+"z500_jul_csf20c.pdf")

In [ ]:
ende